# Getting Started

**Useful Documentation:**
* [Langchain](https://python.langchain.com/docs/)
* [Beautiful Soup](https://beautiful-soup-4.readthedocs.io/en/latest/#quick-start)
* [CCSB GitHub Repo](https://github.com/lgtanimoto/CulturalCSBytes/tree/main)

**Installations:**
```bash
$ pip install beautifulsoup4
$ pip install langchain
$ pip install python-dotenv
$ pip install requests
```

**Imports:**

In [117]:
# built-in python modules
import json
import os
import re
from io import StringIO
from html.parser import HTMLParser
import random
from typing import Dict, List

# langchain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

# python libraries
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import requests

**Set-up OpenAI API Key**:
* generate a secret key in the [openai API key manager](https://platform.openai.com/account/api-keys)
* create a .env file with the environmental variable
```openai_api_key=insert_your_key_here```
* load in the api key by running the following code block

In [2]:
load_dotenv()
openai_api_key = os.getenv("openai_api_key")
llm = OpenAI(model_name="gpt-4", openai_api_key=openai_api_key)
chat_turbo = ChatOpenAI(temperature=0, openai_api_key=openai_api_key, model_name="gpt-3.5-turbo")

c:\Users\aimon\anaconda3\lib\site-packages\langchain\llms\openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\aimon\anaconda3\lib\site-packages\langchain\llms\openai.py:696: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


**Load Example Questions from the [GitHub Repo](https://github.com/lgtanimoto/CulturalCSBytes/tree/main/content/A000/A000/)**

Note: we scrape the 50 default-culture questions to use as examples when generating new questions.

In [3]:
# scrape default questions
URL = "https://github.com/lgtanimoto/CulturalCSBytes/tree/main/content/A000/A000"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

# get all links to CCSB question json files
links = [("https://raw.githubusercontent.com" + link['href'].replace("blob/","")) for link in soup.find_all("a") if ".json" in link['href']]
# list of JSON objects for default questions
default_questions = [json.loads(requests.get(link).text) for link in links]

In [4]:
# Strip HTML from string
class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

# remove unwanted fields from dictionary
def remove_fields(question, fields=['FixedOrderFromBottom', 'AnswerImage', 'QuestionImage']):
    for field in fields:
        if field in question.keys():
            question.pop(field)

Create learning objective list and groupings.

In [5]:
# list of learning standards (objectives), with the full 6-character code
raw_objectives = {
    "1A-CS-01": "Select and operate appropriate software to perform a variety of tasks, and recognize that users have different needs and preferences for the technology they use.",
    "1A-CS-02": "Use appropriate terminology in identifying and describing the function of common physical components of computing systems (hardware).",
    "1A-CS-03": "Describe basic hardware and software problems using accurate terminology.",
    "1A-NI-04": "Explain what passwords are and why we use them, and use strong passwords to protect devices and information from unauthorized access.",
    "1A-DA-05": "Store, copy, search, retrieve, modify, and delete information using a computing device and define the information stored as data.",
    "1A-DA-06": "Collect and present the same data in various visual formats.",
    "1A-DA-07": "Identify and describe patterns in data visualizations, such as charts or graphs, to make predictions.",
    "1A-AP-08": "Model daily processes by creating and following algorithms (sets of step-by-step instructions) to complete tasks.",
    "1A-AP-09": "Model the way programs store and manipulate data by using numbers or other symbols to represent information.",
    "1A-AP-10": "Develop programs with sequences and simple loops, to express ideas or address a problem.",
    "1A-AP-11": "Decompose (break down) the steps needed to solve a problem into a precise sequence of instructions.",
    "1A-AP-12": "Develop plans that describe a program’s sequence of events, goals, and expected outcomes.",
    "1A-AP-13": "Give attribution when using the ideas and creations of others while developing programs.",
    "1A-AP-14": "Debug (identify and fix) errors in an algorithm or program that includes sequences and simple loops.",
    "1A-AP-15": "Using correct terminology, describe steps taken and choices made during the iterative process of program development.",
    "1A-IC-16": "Compare how people live and work before and after the implementation or adoption of new computing technology.",
    "1A-IC-17": "Work respectfully and responsibly with others online.",
    "1A-IC-18": "Keep login information private, and log off of devices appropriately.",
    "1B-CS-01": "Describe how internal and external parts of computing devices function to form a system.",
    "1B-CS-02": "Model how computer hardware and software work together as a system to accomplish tasks.",
    "1B-CS-03": "Determine potential solutions to solve simple hardware and software problems using common troubleshooting strategies.",
    "1B-NI-04": "Model how information is broken down into smaller pieces, transmitted as packets through multiple devices over networks and the Internet, and reassembled at the destination.",
    "1B-NI-05": "Discuss real-world cybersecurity problems and how personal information can be protected.",
    "1B-DA-06": "Organize and present collected data visually to highlight relationships and support a claim.",
    "1B-DA-07": "Use data to highlight or propose cause-and-effect relationships, predict outcomes, or communicate an idea.",
    "1B-AP-08": "Compare and refine multiple algorithms for the same task and determine which is the most appropriate.",
    "1B-AP-09": "Create programs that use variables to store and modify data.",
    "1B-AP-10": "Create programs that include sequences, events, loops, and conditionals.",
    "1B-AP-11": "Decompose (break down) problems into smaller, manageable subproblems to facilitate the program development process.",
    "1B-AP-12": "Modify, remix, or incorporate portions of an existing program into one's own work, to develop something new or add more advanced features.",
    "1B-AP-13": "Use an iterative process to plan the development of a program by including others' perspectives and considering user preferences.",
    "1B-AP-14": "Observe intellectual property rights and give appropriate attribution when creating or remixing programs.",
    "1B-AP-15": "Test and debug (identify and fix errors) a program or algorithm to ensure it runs as intended.",
    "1B-AP-16": "Take on varying roles, with teacher guidance, when collaborating with peers during the design, implementation, and review stages of program development.",
    "1B-AP-17": "Describe choices made during program development using code comments, presentations, and demonstrations.",
    "1B-IC-18": "Discuss computing technologies that have changed the world, and express how those technologies influence, and are influenced by, cultural practices.",
    "1B-IC-19": "Brainstorm ways to improve the accessibility and usability of technology products for the diverse needs and wants of users.",
    "1B-IC-20": "Seek diverse perspectives for the purpose of improving computational artifacts.",
    "1B-IC-21": "Use public domain or creative commons media, and refrain from copying or using material created by others without permission.",
    "2-CS-01": "Recommend improvements to the design of computing devices, based on an analysis of how users interact with the devices.",
    "2-CS-02": "Design projects that combine hardware and software components to collect and exchange data.",
    "2-CS-03": "Systematically identify and fix problems with computing devices and their components.",
    "2-NI-04": "Model the role of protocols in transmitting data across networks and the Internet.",
    "2-NI-05": "Explain how physical and digital security measures protect electronic information.",
    "2-NI-06": "Apply multiple methods of encryption to model the secure transmission of information.",
    "2-DA-07": "Represent data using multiple encoding schemes.",
    "2-DA-08": "Collect data using computational tools and transform the data to make it more useful and reliable.",
    "2-DA-09": "Refine computational models based on the data they have generated.",
    "2-AP-10": "Use flowcharts and/or pseudocode to address complex problems as algorithms.",
    "2-AP-11": "Create clearly named variables that represent different data types and perform operations on their values.",
    "2-AP-12": "Design and iteratively develop programs that combine control structures, including nested loops and compound conditionals.",
    "2-AP-13": "Decompose problems and subproblems into parts to facilitate the design, implementation, and review of programs.",
    "2-AP-14": "Create procedures with parameters to organize code and make it easier to reuse.",
    "2-AP-15": "Seek and incorporate feedback from team members and users to refine a solution that meets user needs.",
    "2-AP-16": "Incorporate existing code, media, and libraries into original programs, and give attribution.",
    "2-AP-17": "Systematically test and refine programs using a range of test cases.",
    "2-AP-18": "Distribute tasks and maintain a project timeline when collaboratively developing computational artifacts.",
    "2-AP-19": "Document programs in order to make them easier to follow, test, and debug.",
    "2-IC-20": "Compare tradeoffs associated with computing technologies that affect people's everyday activities and career options.",
    "2-IC-21": "Discuss issues of bias and accessibility in the design of existing technologies.",
    "2-IC-22": "Collaborate with many contributors through strategies such as crowdsourcing or surveys when creating a computational artifact.",
    "2-IC-23": "Describe tradeoffs between allowing information to be public and keeping information private and secure.",
    "3A-CS-01": "Explain how abstractions hide the underlying implementation details of computing systems embedded in everyday objects.",
    "3A-CS-02": "Compare levels of abstraction and interactions between application software, system software, and hardware layers.",
    "3A-CS-03": "Develop guidelines that convey systematic troubleshooting strategies that others can use to identify and fix errors.",
    "3A-NI-04": "Evaluate the scalability and reliability of networks, by describing the relationship between routers, switches, servers, topology, and addressing.",
    "3A-NI-05": "Give examples to illustrate how sensitive data can be affected by malware and other attacks.",
    "3A-NI-06": "Recommend security measures to address various scenarios based on factors such as efficiency, feasibility, and ethical impacts.",
    "3A-NI-07": "Compare various security measures, considering tradeoffs between the usability and security of a computing system.",
    "3A-NI-08": "Explain tradeoffs when selecting and implementing cybersecurity recommendations.",
    "3A-DA-09": "Translate between different bit representations of real-world phenomena, such as characters, numbers, and images.",
    "3A-DA-10": "Evaluate the tradeoffs in how data elements are organized and where data is stored.",
    "3A-DA-11": "Create interactive data visualizations using software tools to help others better understand real-world phenomena.",
    "3A-DA-12": "Create computational models that represent the relationships among different elements of data collected from a phenomenon or process.",
    "3A-AP-13": "Create prototypes that use algorithms to solve computational problems by leveraging prior student knowledge and personal interests.",
    "3A-AP-14": "Use lists to simplify solutions, generalizing computational problems instead of repeatedly using simple variables.",
    "3A-AP-15": "Justify the selection of specific control structures when tradeoffs involve implementation, readability, and program performance, and explain the benefits and drawbacks of choices made.",
    "3A-AP-16": "Design and iteratively develop computational artifacts for practical intent, personal expression, or to address a societal issue by using events to initiate instructions.",
    "3A-AP-17": "Decompose problems into smaller components through systematic analysis, using constructs such as procedures, modules, and/or objects.",
    "3A-AP-18": "Create artifacts by using procedures within a program, combinations of data and procedures, or independent but interrelated programs.",
    "3A-AP-19": "Systematically design and develop programs for broad audiences by incorporating feedback from users.",
    "3A-AP-20": "Evaluate licenses that limit or restrict use of computational artifacts when using resources such as libraries.",
    "3A-AP-21": "Evaluate and refine computational artifacts to make them more usable and accessible.",
    "3A-AP-22": "Design and develop computational artifacts working in team roles using collaborative tools.",
    "3A-AP-23": "Document design decisions using text, graphics, presentations, and/or demonstrations in the development of complex programs.",
    "3A-IC-24": "Evaluate the ways computing impacts personal, ethical, social, economic, and cultural practices.",
    "3A-IC-25": "Test and refine computational artifacts to reduce bias and equity deficits.",
    "3A-IC-26": "Demonstrate ways a given algorithm applies to problems across disciplines.",
    "3A-IC-27": "Use tools and methods for collaboration on a project to increase connectivity of people in different cultures and career fields.",
    "3A-IC-28": "Explain the beneficial and harmful effects that intellectual property laws can have on innovation.",
    "3A-IC-29": "Explain the privacy concerns related to the collection and generation of data through automated processes that may not be evident to users.",
    "3A-IC-30": "Evaluate the social and economic implications of privacy in the context of safety, law, or ethics.",
    "3B-CS-01": "Categorize the roles of operating system software.",
    "3B-CS-02": "Illustrate ways computing systems implement logic, input, and output through hardware components.",
    "3B-NI-03": "Describe the issues that impact network functionality (e.g., bandwidth, load, delay, topology).",
    "3B-NI-04": "Compare ways software developers protect devices and information from unauthorized access.",
    "3B-DA-05": "Use data analysis tools and techniques to identify patterns in data representing complex systems.",
    "3B-DA-06": "Select data collection tools and techniques to generate data sets that support a claim or communicate information.",
    "3B-DA-07": "Evaluate the ability of models and simulations to test and support the refinement of hypotheses.",
    "3B-AP-08": "Describe how artificial intelligence drives many software and physical systems.",
    "3B-AP-09": "Implement an artificial intelligence algorithm to play a game against a human opponent or solve a problem.",
    "3B-AP-10": "Use and adapt classic algorithms to solve computational problems.",
    "3B-AP-11": "Evaluate algorithms in terms of their efficiency, correctness, and clarity.",
    "3B-AP-12": "Compare and contrast fundamental data structures and their uses.",
    "3B-AP-13": "Illustrate the flow of execution of a recursive algorithm.",
    "3B-AP-14": "Construct solutions to problems using student-created components, such as procedures, modules and/or objects.",
    "3B-AP-15": "Analyze a large-scale computational problem and identify generalizable patterns that can be applied to a solution.",
    "3B-AP-16": "Demonstrate code reuse by creating programming solutions using libraries and APIs.",
    "3B-AP-17": "Plan and develop programs for broad audiences using a software life cycle process.",
    "3B-AP-18": "Explain security issues that might lead to compromised computer programs.",
    "3B-AP-19": "Develop programs for multiple computing platforms.",
    "3B-AP-20": "Use version control systems, integrated development environments (IDEs), and collaborative tools and practices (code documentation) in a group software project.",
    "3B-AP-21": "Develop and use a series of test cases to verify that a program performs according to its design specifications.",
    "3B-AP-22": "Modify an existing program to add additional functionality and discuss intended and unintended implications (e.g., breaking other functionality).",
    "3B-AP-23": "Evaluate key qualities of a program through a process such as a code review.",
    "3B-AP-24": "Compare multiple programming languages and discuss how their features make them suitable for solving different types of problems.",
    "3B-IC-25": "Evaluate computational artifacts to maximize their beneficial effects and minimize harmful effects on society.",
    "3B-IC-26": "Evaluate the impact of equity, access, and influence on the distribution of computing resources in a global society.",
    "3B-IC-27": "Predict how computational innovations that have revolutionized aspects of our culture might evolve.",
    "3B-IC-28": "Debate laws and regulations that impact the development and use of software.",
}

In [6]:
# stores learning objectives in groups accessed by abbreviated one letter group name
group_names = {
    'C': "Computing Systems",
    'N': "Networks and the Internet",
    'D': "Data and Analysis",
    'A': "Algorithms and Programming",
    'I': "Impacts of Computing",
}

lo_code_groups = {
    'C': [],
    'N': [],
    'D': [],
    'A': [],
    'I': [],
}

In [7]:
'''schema conversion:
First character 1A - 1, 1B - 2, 2 - 3, 3A - 4, 3B - 5
Second character is first character after dash
last two characters is the number at the end
'''
lo_list = dict()

level_conversions = {
    "1A": "1",
    "1B": "2",
    "2": "3",
    "3A": "4",
    "3B": "5",
}

# Convert raw objectives' full 6-character codes to the abbreviated 4-character codes and categorize into the lo_code_groups
for k, v in raw_objectives.items():
    code_parts = k.split("-")
    level = code_parts[0]
    lo_group = code_parts[1][0]
    q_num = code_parts[2]

    abbrev_code = level_conversions[level] + lo_group + q_num
    lo_list[abbrev_code] = v
    lo_code_groups[lo_group].append(abbrev_code)


In [8]:
# list of learning standards (objectives) with abbreviated 4-character codes
lo_list = {
    '1C01': 'Select and operate appropriate software to perform a variety of tasks, and recognize that users have different needs and preferences for the technology they use.',
    '1C02': 'Use appropriate terminology in identifying and describing the function of common physical components of computing systems (hardware).',
    '1C03': 'Describe basic hardware and software problems using accurate terminology.',
    '1N04': 'Explain what passwords are and why we use them, and use strong passwords to protect devices and information from unauthorized access.',
    '1D05': 'Store, copy, search, retrieve, modify, and delete information using a computing device and define the information stored as data.',
    '1D06': 'Collect and present the same data in various visual formats.',
    '1D07': 'Identify and describe patterns in data visualizations, such as charts or graphs, to make predictions.',
    '1A08': 'Model daily processes by creating and following algorithms (sets of step-by-step instructions) to complete tasks.',
    '1A09': 'Model the way programs store and manipulate data by using numbers or other symbols to represent information.',
    '1A10': 'Develop programs with sequences and simple loops, to express ideas or address a problem.',
    '1A11': 'Decompose (break down) the steps needed to solve a problem into a precise sequence of instructions.',
    '1A12': 'Develop plans that describe a program’s sequence of events, goals, and expected outcomes.',
    '1A13': 'Give attribution when using the ideas and creations of others while developing programs.',
    '1A14': 'Debug (identify and fix) errors in an algorithm or program that includes sequences and simple loops.',
    '1A15': 'Using correct terminology, describe steps taken and choices made during the iterative process of program development.',
    '1I16': 'Compare how people live and work before and after the implementation or adoption of new computing technology.',
    '1I17': 'Work respectfully and responsibly with others online.',
    '1I18': 'Keep login information private, and log off of devices appropriately.',
    '2C01': 'Describe how internal and external parts of computing devices function to form a system.',
    '2C02': 'Model how computer hardware and software work together as a system to accomplish tasks.',
    '2C03': 'Determine potential solutions to solve simple hardware and software problems using common troubleshooting strategies.',
    '2N04': 'Model how information is broken down into smaller pieces, transmitted as packets through multiple devices over networks and the Internet, and reassembled at the destination.',
    '2N05': 'Discuss real-world cybersecurity problems and how personal information can be protected.',
    '2D06': 'Organize and present collected data visually to highlight relationships and support a claim.',
    '2D07': 'Use data to highlight or propose cause-and-effect relationships, predict outcomes, or communicate an idea.',
    '2A08': 'Compare and refine multiple algorithms for the same task and determine which is the most appropriate.',
    '2A09': 'Create programs that use variables to store and modify data.',
    '2A10': 'Create programs that include sequences, events, loops, and conditionals.',
    '2A11': 'Decompose (break down) problems into smaller, manageable subproblems to facilitate the program development process.',
    '2A12': "Modify, remix, or incorporate portions of an existing program into one's own work, to develop something new or add more advanced features.",
    '2A13': "Use an iterative process to plan the development of a program by including others' perspectives and considering user preferences.",
    '2A14': 'Observe intellectual property rights and give appropriate attribution when creating or remixing programs.',
    '2A15': 'Test and debug (identify and fix errors) a program or algorithm to ensure it runs as intended.',
    '2A16': 'Take on varying roles, with teacher guidance, when collaborating with peers during the design, implementation, and review stages of program development.',
    '2A17': 'Describe choices made during program development using code comments, presentations, and demonstrations.',
    '2I18': 'Discuss computing technologies that have changed the world, and express how those technologies influence, and are influenced by, cultural practices.',
    '2I19': 'Brainstorm ways to improve the accessibility and usability of technology products for the diverse needs and wants of users.',
    '2I20': 'Seek diverse perspectives for the purpose of improving computational artifacts.',
    '2I21': 'Use public domain or creative commons media, and refrain from copying or using material created by others without permission.',
    '3C01': 'Recommend improvements to the design of computing devices, based on an analysis of how users interact with the devices.',
    '3C02': 'Design projects that combine hardware and software components to collect and exchange data.',
    '3C03': 'Systematically identify and fix problems with computing devices and their components.',
    '3N04': 'Model the role of protocols in transmitting data across networks and the Internet.',
    '3N05': 'Explain how physical and digital security measures protect electronic information.',
    '3N06': 'Apply multiple methods of encryption to model the secure transmission of information.',
    '3D07': 'Represent data using multiple encoding schemes.',
    '3D08': 'Collect data using computational tools and transform the data to make it more useful and reliable.',
    '3D09': 'Refine computational models based on the data they have generated.',
    '3A10': 'Use flowcharts and/or pseudocode to address complex problems as algorithms.',
    '3A11': 'Create clearly named variables that represent different data types and perform operations on their values.',
    '3A12': 'Design and iteratively develop programs that combine control structures, including nested loops and compound conditionals.',
    '3A13': 'Decompose problems and subproblems into parts to facilitate the design, implementation, and review of programs.',
    '3A14': 'Create procedures with parameters to organize code and make it easier to reuse.',
    '3A15': 'Seek and incorporate feedback from team members and users to refine a solution that meets user needs.',
    '3A16': 'Incorporate existing code, media, and libraries into original programs, and give attribution.',
    '3A17': 'Systematically test and refine programs using a range of test cases.',
    '3A18': 'Distribute tasks and maintain a project timeline when collaboratively developing computational artifacts.',
    '3A19': 'Document programs in order to make them easier to follow, test, and debug.',
    '3I20': "Compare tradeoffs associated with computing technologies that affect people's everyday activities and career options.",
    '3I21': 'Discuss issues of bias and accessibility in the design of existing technologies.',
    '3I22': 'Collaborate with many contributors through strategies such as crowdsourcing or surveys when creating a computational artifact.',
    '3I23': 'Describe tradeoffs between allowing information to be public and keeping information private and secure.',
    '4C01': 'Explain how abstractions hide the underlying implementation details of computing systems embedded in everyday objects.',
    '4C02': 'Compare levels of abstraction and interactions between application software, system software, and hardware layers.',
    '4C03': 'Develop guidelines that convey systematic troubleshooting strategies that others can use to identify and fix errors.',
    '4N04': 'Evaluate the scalability and reliability of networks, by describing the relationship between routers, switches, servers, topology, and addressing.',
    '4N05': 'Give examples to illustrate how sensitive data can be affected by malware and other attacks.',
    '4N06': 'Recommend security measures to address various scenarios based on factors such as efficiency, feasibility, and ethical impacts.',
    '4N07': 'Compare various security measures, considering tradeoffs between the usability and security of a computing system.',
    '4N08': 'Explain tradeoffs when selecting and implementing cybersecurity recommendations.',
    '4D09': 'Translate between different bit representations of real-world phenomena, such as characters, numbers, and images.',
    '4D10': 'Evaluate the tradeoffs in how data elements are organized and where data is stored.',
    '4D11': 'Create interactive data visualizations using software tools to help others better understand real-world phenomena.',
    '4D12': 'Create computational models that represent the relationships among different elements of data collected from a phenomenon or process.',
    '4A13': 'Create prototypes that use algorithms to solve computational problems by leveraging prior student knowledge and personal interests.',
    '4A14': 'Use lists to simplify solutions, generalizing computational problems instead of repeatedly using simple variables.',
    '4A15': 'Justify the selection of specific control structures when tradeoffs involve implementation, readability, and program performance, and explain the benefits and drawbacks of choices made.',
    '4A16': 'Design and iteratively develop computational artifacts for practical intent, personal expression, or to address a societal issue by using events to initiate instructions.',
    '4A17': 'Decompose problems into smaller components through systematic analysis, using constructs such as procedures, modules, and/or objects.',
    '4A18': 'Create artifacts by using procedures within a program, combinations of data and procedures, or independent but interrelated programs.',
    '4A19': 'Systematically design and develop programs for broad audiences by incorporating feedback from users.',
    '4A20': 'Evaluate licenses that limit or restrict use of computational artifacts when using resources such as libraries.',
    '4A21': 'Evaluate and refine computational artifacts to make them more usable and accessible.',
    '4A22': 'Design and develop computational artifacts working in team roles using collaborative tools.',
    '4A23': 'Document design decisions using text, graphics, presentations, and/or demonstrations in the development of complex programs.',
    '4I24': 'Evaluate the ways computing impacts personal, ethical, social, economic, and cultural practices.',
    '4I25': 'Test and refine computational artifacts to reduce bias and equity deficits.',
    '4I26': 'Demonstrate ways a given algorithm applies to problems across disciplines.',
    '4I27': 'Use tools and methods for collaboration on a project to increase connectivity of people in different cultures and career fields.',
    '4I28': 'Explain the beneficial and harmful effects that intellectual property laws can have on innovation.',
    '4I29': 'Explain the privacy concerns related to the collection and generation of data through automated processes that may not be evident to users.',
    '4I30': 'Evaluate the social and economic implications of privacy in the context of safety, law, or ethics.',
    '5C01': 'Categorize the roles of operating system software.',
    '5C02': 'Illustrate ways computing systems implement logic, input, and output through hardware components.',
    '5N03': 'Describe the issues that impact network functionality (e.g., bandwidth, load, delay, topology).',
    '5N04': 'Compare ways software developers protect devices and information from unauthorized access.',
    '5D05': 'Use data analysis tools and techniques to identify patterns in data representing complex systems.',
    '5D06': 'Select data collection tools and techniques to generate data sets that support a claim or communicate information.',
    '5D07': 'Evaluate the ability of models and simulations to test and support the refinement of hypotheses.',
    '5A08': 'Describe how artificial intelligence drives many software and physical systems.',
    '5A09': 'Implement an artificial intelligence algorithm to play a game against a human opponent or solve a problem.',
    '5A10': 'Use and adapt classic algorithms to solve computational problems.',
    '5A11': 'Evaluate algorithms in terms of their efficiency, correctness, and clarity.',
    '5A12': 'Compare and contrast fundamental data structures and their uses.',
    '5A13': 'Illustrate the flow of execution of a recursive algorithm.',
    '5A14': 'Construct solutions to problems using student-created components, such as procedures, modules and/or objects.',
    '5A15': 'Analyze a large-scale computational problem and identify generalizable patterns that can be applied to a solution.',
    '5A16': 'Demonstrate code reuse by creating programming solutions using libraries and APIs.',
    '5A17': 'Plan and develop programs for broad audiences using a software life cycle process.',
    '5A18': 'Explain security issues that might lead to compromised computer programs.',
    '5A19': 'Develop programs for multiple computing platforms.',
    '5A20': 'Use version control systems, integrated development environments (IDEs), and collaborative tools and practices (code documentation) in a group software project.',
    '5A21': 'Develop and use a series of test cases to verify that a program performs according to its design specifications.',
    '5A22': 'Modify an existing program to add additional functionality and discuss intended and unintended implications (e.g., breaking other functionality).',
    '5A23': 'Evaluate key qualities of a program through a process such as a code review.',
    '5A24': 'Compare multiple programming languages and discuss how their features make them suitable for solving different types of problems.',
    '5I25': 'Evaluate computational artifacts to maximize their beneficial effects and minimize harmful effects on society.',
    '5I26': 'Evaluate the impact of equity, access, and influence on the distribution of computing resources in a global society.',
    '5I27': 'Predict how computational innovations that have revolutionized aspects of our culture might evolve.',
    '5I28': 'Debate laws and regulations that impact the development and use of software.'
}

Create examples list.

In [9]:
# few-shot example string for topic extraction from default questions
topic_extraction_examples = """Your job is to extract the general topic of the following JSON-formatted question in three words or less without focusing on its technical aspects. The following are some examples, 
Input: '''{
    'Question': '<p><span class="qname">Chris</span> is eating cold cereal for breakfast before school.   Which of the following best describes the <em>algorithm</em> <span class="qname">Chris</span> should use to make breakfast?</p>',
    'Answer1': 'Cereal, Milk, Bowl, Spoon',
    'Answer2': 'After waking up, ask parent to make cornflakes for breakfast',
    'Answer3': 'Get out bowl and spoon.  Pour cereal out of box into bowl. Pour milk into bowl. Eat with spoon until cereal and milk are gone.',
    'Answer4': 'Eat.  Put pot on stove, turn on stove, and boil for seven minutes. Turn off stove.  Puts two eggs in pot.  Put water in pot.',
    'CorrectAnswer': 3,
    'Explanation': '<p>An <em>algorithm</em> is a step-by-step series of instructions to accomplish a given task.</p>',
}'''
Output: "Making Breakfast"

Input: '''{
    'Question': '<p>&quot;\'Boolean\', that\'s a funny name.   Is it a kind of bean?&quot; asked <span class="qname">Wisdom</span> in their computer science class.</p><p>&quot;No,&quot; replied <span class="qname">Wisdom</span>\'s teacher.  &quot;A \'Boolean\' is a variable representing yes or no, true or false, on or off...&quot;</p><p>What is another representation of a <em>Boolean</em> that <span class="qname">Wisdom</span>\'s teacher could have added to the explanation?</p>',
    'Answer1': "'A', 'B', 'C', or 'D'",
    'Answer2': '0 or 1',
    'Answer3': 'The text strings &quot;true&quot; or &quot;false&quot;',
    'Answer4': 'The temperature of an object',
    'CorrectAnswer': 2,
    'Explanation': '<p><em>Boolean</em> variables are often stored as binary 0 or 1 internally within a computer program.   Boolean variables can only have two possible values.  String values which can store many possible values cannot be Boolean variables.</p>',
}'''
Output: "Teacher - student conversation"

Input: '''{
    'Question': 'Kai has the following metal image for an icon for their club to use at several school events.  Kai has a drawing program with a programming language that make it easy to draw regular polygons in multiple colors.    How should Kai decompose or break down the generation of this image into a small number of steps?',
    'Answer1': 'Loop through each pixel in the image and set the color as appropriate',
    'Answer2': 'Draw the borders of each figure in the image and then paint each resulting shape with the appropriate color using the paint can function',
    'Answer3': 'Develop software functions that can draw polygons filled with any color',
    'Answer4': 'Draw a blue circle using the programming language in the drawing program.  Then draw a red hexagon and a green triangle using same programming language',
    'Explanation': 'Decomposition is breaking down the steps needed to solve a problem into a precise sequence of instructions.  Programming languages that support drawing have functions that can draw circles and other polygons and fill them with a color.   While the other solutions do break down the problem somewhat, each instruction in the sequence is unclear. '
}'''
Output: "School art project"
"""

In [10]:
### Create Examples List

examples = []

for i, dq in enumerate(default_questions):
    q = dq['QuestionJSON']

    q['CorrectAnswer'] = str(q['CorrectAnswer'])

    remove_fields(q)
    for k in q.keys():
        q[k] = strip_tags(q[k])

    example = {
        'MQCode': dq['MQCode'],
        'learning_objective': lo_list[dq['MQCode']],
        'topic': None,
        'question_str': json.dumps(q, indent=4)
    }

    topic = chat_turbo([
        SystemMessage(content=topic_extraction_examples),
        HumanMessage(content=json.dumps(q, indent=4))
    ]).content
    
    print(i, topic)
    example['topic'] = topic

    examples.append(example)

0 Making Breakfast
1 Boolean representation
2 Programming a robot
3 Drawing Icon
4 Drawing shapes
5 Promotional website photo
6 Fixing temperature program
7 Data Analysis
8 Insufficient RAM
9 Computing data sources
10 Data visualization
11 Water bottle sales
12 Poll methodology and conclusions
13 School changes 2019-2022
14 Collaborative work disagreement
15 School email incident
16 "Choosing a password"
17 "Variable in program"
18 Variables and output
19 Making a sandwich
20 Planting flowers
21 Function explanation
22 Computer Input and Output
23 Ice Cream Satisfaction
24 Cultural Misunderstanding
25 Online menu accessibility
26 Video streaming process
27 Phishing attempt
28 Robot maze navigation
29 Animal counting program
30 Checkerboard design
31 World Cup song
32 Creating repeated characters
33 Temperature classification
34 Binary to hexadecimal conversion
35 Popsicle temperature prediction
36 Future police officers
37 Code confidentiality
38 Two-factor authentication
39 Caesar cip

In [122]:
### Full List of Examples. (result of previous cell)

examples = [
 {'MQCode': '1A08',
  'learning_objective': 'Model daily processes by creating and following algorithms (sets of step-by-step instructions) to complete tasks.',
  'topic': 'Making Breakfast',
  'question_str': '{\n    "Question": "Chris is eating cold cereal for breakfast before school.   Which of the following best describes the algorithm Chris should use to make breakfast?",\n    "Answer1": "Cereal, Milk, Bowl, Spoon",\n    "Answer2": "After waking up, ask parent to make cornflakes for breakfast",\n    "Answer3": "Get out bowl and spoon.  Pour cereal out of box into bowl. Pour milk into bowl. Eat with spoon until cereal and milk are gone.",\n    "Answer4": "Eat.  Put pot on stove, turn on stove, and boil for seven minutes. Turn off stove.  Puts two eggs in pot.  Put water in pot.",\n    "CorrectAnswer": "3",\n    "Explanation": "An algorithm is a step-by-step series of instructions to accomplish a given task."\n}'},
 {'MQCode': '1A09',
  'learning_objective': 'Model the way programs store and manipulate data by using numbers or other symbols to represent information.',
  'topic': 'Boolean representation',
  'question_str': '{\n    "Question": "\\"\'Boolean\', that\'s a funny name.   Is it a kind of bean?\\" asked Wisdom in their computer science class.\\"No,\\" replied Wisdom\'s teacher.  \\"A \'Boolean\' is a variable representing yes or no, true or false, on or off...\\"What is another representation of a Boolean that Wisdom\'s teacher could have added to the explanation?",\n    "Answer1": "\'A\', \'B\', \'C\', or \'D\'",\n    "Answer2": "0 or 1",\n    "Answer3": "The text strings \\"true\\" or \\"false\\"",\n    "Answer4": "The temperature of an object",\n    "CorrectAnswer": "2",\n    "Explanation": "Boolean variables are often stored as binary 0 or 1 internally within a computer program.   Boolean variables can only have two possible values.  String values which can store many possible values cannot be Boolean variables."\n}'},
 {'MQCode': '1A10',
  'learning_objective': 'Develop programs with sequences and simple loops, to express ideas or address a problem.',
  'topic': 'Programming a robot',
  'question_str': '{\n    "Question": "Rory is programming a humanoid robot to get off the sofa, walk to the light switch, and turn off the lights in a room.  Which of the following most closely represents the code that Rory needs to write:",\n    "Answer1": "StandUp()TurnToward(lightSwitch)repeat while not= AtLocation(lightSwitch)RightWalk(1)LeftWalk(1)endTurnOffLight()",\n    "Answer2": "TurnOffLight()repeat while not= AtLocation(lightSwitch)RightWalk(1)LeftWalk(1)endTurnToward(lightSwitch)StandUp()",\n    "Answer3": "StandUp()TurnToward(lightSwitch)repeat 1000 timesRightWalk(1)endLeftWalk(1)TurnOffLight()",\n    "Answer4": "StandUp()TurnToward(lightSwitch)repeat while AtLocation(lightSwitch)RightWalk(1)LeftWalk(1)TurnOffLight()end",\n    "CorrectAnswer": "1",\n    "Explanation": "First, robot stands up so it can walk.  Second, robot turns toward the light switch so it walks in the correct direction.  Third, the robot walks right-left until it reaches the light switch.  Finally, the robot turns off the light. "\n}'},
 {'MQCode': '1A11',
  'learning_objective': 'Decompose (break down) the steps needed to solve a problem into a precise sequence of instructions.',
  'topic': 'Drawing Icon',
  'question_str': '{\n    "Question": "Kai has the following metal image for an icon for their club to use at several school events.  Kai has a drawing program with a programming language that make it easy to draw regular polygons in multiple colors.    How should Kai decompose or break down the generation of this image into a small number of steps?",\n    "Answer1": "Loop through each pixel in the image and set the color as appropriate",\n    "Answer2": "Draw the borders of each figure in the image and then paint each resulting shape with the appropriate color using the paint can function",\n    "Answer3": "Develop software functions that can draw polygons filled with any color",\n    "Answer4": "Draw a blue circle using the programming language in the drawing program.  Then draw a red hexagon and a green triangle using same programming language",\n    "CorrectAnswer": "4",\n    "Explanation": "Decomposition is breaking down the steps needed to solve a problem into a precise sequence of instructions.  Programming languages that support drawing have functions that can draw circles and other polygons and fill them with a color.   While the other solutions do break down the problem somewhat, each instruction in the sequence is unclear. "\n}'},
 {'MQCode': '1A12',
  'learning_objective': 'Develop plans that describe a program’s sequence of events, goals, and expected outcomes.',
  'topic': 'Drawing shapes',
  'question_str': '{\n    "Question": "At school, Chile uses a programming language with code statements to draw lines, circles, triangles, and rectangles filled with 16 different colors on a 400-by-400-pixel grid. Chile can write programs that combine these code statements with variables, loops, conditionals, and procedures. Based on this description, which one of the following could NOT be created with this programming language?",\n    "Answer1": "A 8 by 8 checkerboard",\n    "Answer2": "A hexagon created from 6 triangles with different colors",\n    "Answer3": "A program tracking the birthday of every student in class",\n    "Answer4": "A pyramid of 10 circles stacked upon each other 4 - 3 - 2 - 1",\n    "CorrectAnswer": "3",\n    "Explanation": "Different programming languages or libraries of programming languages have different strengths and uses.   Use the right programming language or library for the right task.  The programming language that Chile uses provides basic drawing functionality but may not be of any help in tracking student birthdays.   A programming language that provided simple spreadsheet or database functionality would be more useful for tracking birthdays. "\n}'},
 {'MQCode': '1A13',
  'learning_objective': 'Give attribution when using the ideas and creations of others while developing programs.',
  'topic': 'Promotional website photo',
  'question_str': '{\n    "Question": "When creating a promotional website for a friend\'s band, Luca finds a professional, close-up photo on another friend\'s Instagram of a famous musician with over ten billion song views on YouTube.   Luca wants to use the Instagram photo to show how the famous musician has inspired many of the songs performed by the band.   What should Luca do?",\n    "Answer1": "Avoid using the photo because the copyright owner of the photo is unknown",\n    "Answer2": "Download the photo from Instagram and post it to the band\'s website without attribution.  The copyright owner is unknown, and the photo is of a famous person",\n    "Answer3": "Download the photo from Instagram and post it to the band\'s website with attribution by providing a link to Instagram.",\n    "Answer4": "Take a screen snapshot of the photo from Instagram and save it.   You now own the copyright to the photo so can post it to the band website without attribution.",\n    "CorrectAnswer": "1",\n    "Explanation": "Because the photo is a close-up professional photo of a famous person, it is highly unlikely that the friend is the copyright owner of the photo.  Instagram is also not the copyright owner. You should only use photos you have taken, a person has given you permission to use, or where copyright, usage, and attribution information are clear. "\n}'},
 {'MQCode': '1A14',
  'learning_objective': 'Debug (identify and fix) errors in an algorithm or program that includes sequences and simple loops.',
  'topic': 'Fixing temperature program',
  'question_str': '{\n    "Question": "Jude is writing a program to find the coldest temperature in the previous year.  Jude has a list of the low temperature for each day that looks as follows:23.521.743.0\\u2026\\u202635.9The original program runs but gives the wrong answer:What change should Jude make to fix the program and output the coldest temperature?",\n    "Answer1": "Modify line 4: if dayCold > maxCold",\n    "Answer2": "Modify line 4: if dayCold = maxCold",\n    "Answer3": "Modify line 5: dayCold = maxCold",\n    "Answer4": "Modify line 1: number maxCold = 1000.0",\n    "CorrectAnswer": "4",\n    "Explanation": "Because maxCold is set so low in line 1, line 4 is never true and maxCold will always be -1000.0.   By modifying line 1 to maxCold = 1000.0 - higher than any expected value, the first value read from the list will become the initial maxCold and the program will work as expected."\n}'},
 {'MQCode': '1C01',
  'learning_objective': 'Select and operate appropriate software to perform a variety of tasks, and recognize that users have different needs and preferences for the technology they use.',
  'topic': 'Data Analysis',
  'question_str': '{\n    "Question": "The students in Pat\'s science class are trying to determine how the weight of a marble affects its speed down a curved and twisted ramp.  They have 25 marbles to test.   What type of software should the students use to track this information and start analyzing this information?",\n    "Answer1": "Word processor software (e.g., Google Docs, Microsoft Word)",\n    "Answer2": "Spreadsheet software (e.g., Google Sheets, Microsoft Excel)",\n    "Answer3": "Social media software (e.g., Facebook, TikTok)",\n    "Answer4": "Internet browser software (Google Chrome, Apple Safari)",\n    "CorrectAnswer": "2",\n    "Explanation": "Spreadsheet software provides computation, organization, analysis, charting, and storage of data tables.  Of the choices provided, only spreadsheet software has the ability to process and chart numerical data."\n}'},
 {'MQCode': '1C03',
  'learning_objective': 'Describe basic hardware and software problems using accurate terminology.',
  'topic': 'Insufficient RAM',
  'question_str': '{\n    "Question": "When Rowan runs one or two applications on a school laptop, these applications run fast even if the applications need to retrieve big files from the Internet.   However, if Rowan has several applications and several browser windows and tabs open at the same time, everything seems to slow down.  Where is the likely source of the bottleneck causing this slowdown?",\n    "Answer1": "A computer virus",\n    "Answer2": "A slow remote Internet server",\n    "Answer3": "A slow graphics card",\n    "Answer4": "A lack of RAM (Random Access Memory)",\n    "CorrectAnswer": "4",\n    "Explanation": "A lack of RAM (Random Access Memory) is the most likely culprit as every application, browser window/tab that is open uses up some RAM even if it is not active.   But computer performance is often tricky to debug.   A computer virus that uses up system resources, but whose effects are not evident until many applications are open is another possibility.   A slow remote Internet server or graphics card may affect a particular application and indirectly the entire system, but the type of application running is not mentioned in this question."\n}'},
 {'MQCode': '1D05',
  'learning_objective': 'Store, copy, search, retrieve, modify, and delete information using a computing device and define the information stored as data.',
  'topic': 'Computing data sources',
  'question_str': '{\n    "Question": "Java wonders about data.   \\"I know can find data when I open an Excel workbook or Google Sheets, but where else can I find computing data?\\"Sumatra replies, \\"Really, all information stored or used by a computer is data.   This includes web pages or streaming video.\\"Which of the following could Sumatra NOT include as an example of computer data?",\n    "Answer1": "Google Chrome or Apple Safari browser code",\n    "Answer2": "CPU (Central processing unit)",\n    "Answer3": "Downloaded music files",\n    "Answer4": "Word processing documents for homework",\n    "CorrectAnswer": "2",\n    "Explanation": "All information stored or used by a computer is data.  One of the innovations of computing is that even computing code is data.   Of the above responses, only the CPU is hardware that uses data rather than data that is being used. "\n}'},
 {'MQCode': '1D06',
  'learning_objective': 'Collect and present the same data in various visual formats.',
  'topic': 'Data visualization',
  'question_str': '{\n    "Question": "After asking each member of the track team, Juneau determined that 5 had birthdays in spring, 10 in summer, 8 in fall, and 6 in winter.   Which of the following are appropriate charts to visualize this information? ",\n    "Answer1": "Bar chart only",\n    "Answer2": "Pie chart only",\n    "Answer3": "Line chart only",\n    "Answer4": "Both the bar chart and the pie chart",\n    "CorrectAnswer": "4",\n    "Explanation": "Both the bar chart and pie chart are appropriate for visualizing categorical data.   The line chart implies that each point happened in sequence.  But because team members may be born in different years, there is none."\n}'},
 {'MQCode': '1D07',
  'learning_objective': 'Identify and describe patterns in data visualizations, such as charts or graphs, to make predictions.',
  'topic': 'Water bottle sales',
  'question_str': '{\n    "Question": "Drew is tracking the number of water bottles sold at soccer games against the temperature at the start of the game.  After 9 games, Drew has created the charts below.   Knowing that today\'s temperature at game time will be 85 degrees, how many bottles of water should Drew expect to sell?",\n    "Answer1": "70 bottles",\n    "Answer2": "45 bottles",\n    "Answer3": "80 bottles",\n    "Answer4": "90 bottles",\n    "CorrectAnswer": "3",\n    "Explanation": "The dotted blue trend line in the top graph crosses 80 bottles at 85 degrees F.   While the bar chart at the bottom contains valid data, it is very difficult to use for prediction purposes."\n}'},
 {'MQCode': '1D07',
  'learning_objective': 'Identify and describe patterns in data visualizations, such as charts or graphs, to make predictions.',
  'topic': 'Poll methodology and conclusions',
  'question_str': '{\n    "Question": "Kauai is writing an article about the best movies of the year for their school newspaper.  To write the article, Kauai created a poll asking students to choose between Movie A, Movie B, Movie C, or Other for \\"Best Movie.\\"  Kauai then asked four friends from their advanced coding class to each ask ten of their friends to answer the poll for a total of 40 responses.  The poll results are as follows:In the article, Kauai included the poll results, the poll methodology, and declared that Movie B was the best movie of the year.   However, soon after the article was published, Kauai received several negative comments from both students and teachers about both the methodology and conclusions of the poll.  Kauai\'s high school is an urban, multi-ethnic, 9-12 public high school with about 1,600 studentsWhich of the following reasons is LEAST LIKELY to be included in the negative comments from TEACHERS about the poll and the article?",\n    "Answer1": "To prevent errors in recording student responses, an online survey tool like Google Forms should have been used.",\n    "Answer2": "Considering that less than 3% of students were polled, the results were too close to make a definitive statement.",\n    "Answer3": "There is an important distinction between \\"Best\\" and \\"Favorite\\".  The informal poll was more about popularity than quality.",\n    "Answer4": "The students polled by Kauai\\ufffds friends in advanced coding class may not be representative of the demographics of the students in Kauai\\ufffds high school.",\n    "CorrectAnswer": "1",\n    "Explanation": "An online survey tool would make it easier to get more responses and would accurately record the responses.  But as only 40 simple responses were recorded, doing this manually is not a significant problem with this poll. However, it is very difficult to create a scientific, representative poll. The students polled are likely to reflect the students in Kauai\'s advanced coding class \\ufffd not the entire school.   And considering the small number of students polled, the results are too close \\ufffd one vote change could result in a three-way tie \\ufffd to definitively state that Movie B was the favorite of all students in the school. "\n}'},
 {'MQCode': '1I16',
  'learning_objective': 'Compare how people live and work before and after the implementation or adoption of new computing technology.',
  'topic': 'School changes 2019-2022',
  'question_str': '{\n    "Question": "Phoenix is writing a short essay on how school has changed between 2019 and 2022, and interviews the business club president, Atlanta, who is three years older.  Due to the COVID-19 pandemic, Phoenix and Atlanta spent much of 2020 and some of 2021 in online or hybrid classrooms.    Which of the following phenomena could Phoenix NOT use to describe as a difference between 2019 and 2022?",\n    "Answer1": "In 2019, essays had to be printed out and turned in.   In 2022, essays could be turned in online",\n    "Answer2": "In 2019, teachers often had to print out reading assignments to hand out to students.  In 2022, teachers either provided scanned copies or links to resources on the Internet.",\n    "Answer3": "In 2019, only two guest speakers visited the business club in person.   In 2022, six guest speakers visited; but were all online except one.",\n    "Answer4": "In 2019, almost all science experiments were done using computer simulations.  In 2022, almost all science experiments were done using physical materials.",\n    "CorrectAnswer": "4",\n    "Explanation": "As many science experiments that were done with physical materials in 2019 could only be done via simulations during virtual and hybrid classes, some of these simulated experiments continued to be used even after physical classes returned. "\n}'},
 {'MQCode': '1I17',
  'learning_objective': 'Work respectfully and responsibly with others online.',
  'topic': 'Collaborative work disagreement',
  'question_str': '{\n    "Question": "London, Sydney, and Cairo are working on a shared presentation for an assignment due the next day.   Each student is working on the presentation online from home, and they are using a video conferencing system to talk with each other.  London notices that one of the slides that Sydney worked on has several statements that London believes are factual errors.Which of the following would NOT be considered respectful and responsible behavior in this situation?",\n    "Answer1": "London fixes the \\"errors\\" quickly when Sydney is taking a bathroom break so that Sydney\\ufffds feelings are not hurt ",\n    "Answer2": "London gets Sydney and Cairo\\ufffds attention on the video call to point out the slide with \\"errors\\" and asks whether they would like to discuss the slide now or later.",\n    "Answer3": "London works on some other slides with the intent of discussing the slide with \\"errors\\" later with Sydney and Cairo.",\n    "Answer4": "London starts researching the \\"errors\\" in question to verify suspicions.",\n    "CorrectAnswer": "1",\n    "Explanation": "If there are differences in opinions on shared work, these differences should be openly and respectfully discussed and resolved among all team members. "\n}'},
 {'MQCode': '1I18',
  'learning_objective': 'Keep login information private, and log off of devices appropriately.',
  'topic': 'School email incident',
  'question_str': '{\n    "Question": "Red is angry.  Yesterday after class, someone e-mailed the following message to Sky using Red\\ufffds school account:Red does not have a crush on Sky and does not remember sending the message.  Which of the following is the most likely explanation for the message?",\n    "Answer1": "Red has a crush on Sky and sent the e-mail to Sky while sleepwalking the previous night.",\n    "Answer2": "Red changed the password for their school account yesterday.",\n    "Answer3": "Red often forgets to log off from the computer at the end of class.   Someone noticed yesterday and sent the mail using Red\'s account.",\n    "Answer4": "Red clicked on an e-mail with the subject line \\"Nigerian prince wants money\\" but realized it was a joke sent by their best friend Moss. ",\n    "CorrectAnswer": "3",\n    "Explanation": "Keep your login information private, and remember to log off of devices appropriately."\n}'},
 {'MQCode': '1N04',
  'learning_objective': 'Explain what passwords are and why we use them, and use strong passwords to protect devices and information from unauthorized access.',
  'topic': '"Choosing a password"',
  'question_str': '{\n    "Question": "Jordan is watching a video on the importance of good passwords to protect your personal information.   Two important aspects of a good password are length and complexity.  Which of the following is the best password for Jordan to use for their accounts on school computers?",\n    "Answer1": "!dadsPizzaisthegr8est",\n    "Answer2": "password",\n    "Answer3": "808-555-1212 (note: student\'s cell phone number",\n    "Answer4": "@^0*)zeq(&%#!$",\n    "CorrectAnswer": "4",\n    "Explanation": "!dadsPizzaisthegr8est is the longest, reasonably complex, and  relatively easy for Jordan to remember.  @^0*)zeq(&%#!$ is too difficult to remember without writing down or using a password manager.   The others are too easy or contain other personal information."\n}'},
 {'MQCode': '2A09',
  'learning_objective': 'Create programs that use variables to store and modify data.',
  'topic': '"Variable in program"',
  'question_str': '{\n    "Question": "Farmer wrote the following program to double the number of apples they want to give to the school.   What is the variable in this program?integer apples = 10apples = apples * 2output \\"Number of apples: \\" + apples",\n    "Answer1": "output",\n    "Answer2": "apples",\n    "Answer3": "integer",\n    "Answer4": "* 2",\n    "CorrectAnswer": "2",\n    "Explanation": "A standard definition of a variable is a \\"value that can change, depending on conditions or information passed to the program\\".  In the example code, apples meets this definition "\n}'},
 {'MQCode': '2A10',
  'learning_objective': 'Create programs that include sequences, events, loops, and conditionals.',
  'topic': 'Variables and output',
  'question_str': '{\n    "Question": "To better understand how variables and programming work, Summit creates the following code:What does this code output on line 4?",\n    "Answer1": "\\u00a0\\u00a03\\u00a0\\u00a0",\n    "Answer2": "\\u00a0\\u00a05\\u00a0\\u00a0",\n    "Answer3": "\\u00a0\\u00a06\\u00a0\\u00a0",\n    "Answer4": "\\u00a0\\u00a08\\u00a0\\u00a0",\n    "CorrectAnswer": "4",\n    "Explanation": "Use code tracing to understand the value of each expression at each step.Line 1a=3Line 2a=3, b=3Line 3a=5, b=3Line 4a+b = 8"\n}'},
 {'MQCode': '2A11',
  'learning_objective': 'Decompose (break down) problems into smaller, manageable subproblems to facilitate the program development process.',
  'topic': 'Making a sandwich',
  'question_str': '{\n    "Question": "River is making a peanut butter and jelly sandwich for lunch.   Which of the following is an algorithm that decomposes the steps to make a peanut butter and jelly sandwich into a precise sequence of instructions?",\n    "Answer1": "Peanut butter, grape jelly, and two pieces of bread",\n    "Answer2": "for breakfast, make fried eggsfor lunch, make peanut butter and jelly sandwichfor dinner, bake a pizzafor dessert, buy some ice cream",\n    "Answer3": "Get the bread, peanut butter, and grape jelly out of the refrigeratorEat the sandwich ",\n    "Answer4": "Get the bread, peanut butter, and grape jelly out of the refrigeratorOpen all containersTake two pieces of bread out and lay them on a counterSpread peanut butter on the top of one piece of bread and grape jelly on the other.Put the two pieces of bread together with the peanut butter and jelly on the inside",\n    "CorrectAnswer": "4",\n    "Explanation": "Decomposition is breaking a process down into well-defined steps.   Answer D is the answer that best breaks down how to make a peanut butter and jelly sandwich"\n}'},
 {'MQCode': '2A15',
  'learning_objective': 'Test and debug (identify and fix errors) a program or algorithm to ensure it runs as intended.',
  'topic': 'Planting flowers',
  'question_str': '{\n    "Question": "Pacific is programming a robot to plant flowers in a single row in a flower bed.  Each flower should be separated by 20 centimeters.  Each flower bed is a minimum of 40 cm and has length that is a multiple of 20 cm.  Pacific\'s initial code is as follows:findLeftEdgeOfFlowerBed()repeat while not= atRightEdgeOfFlowerBed()plantFlower()moveRight(20)endHowever, the initial code attempts to plant a flower in the wall of the left edge.   Which of the following code snippets correctly implements this algorithm?",\n    "Answer1": "findLeftEdgeOfFlowerBed()repeat while not= atRightEdgeOfFlowerBed()moveRight(20)plantFlower()end",\n    "Answer2": "findLeftEdgeOfFlowerBed()moveRight(20)repeat while not= atRightEdgeOfFlowerBed()plantFlower()moveRight(20)end",\n    "Answer3": "findLeftEdgeOfFlowerBed()repeat until atRightEdgeOfFlowerBed()plantFlower()moveRight(20)end",\n    "Answer4": "findLeftEdgeOfFlowerBed()repeat until atRightEdgeOfFlowerBed()moveRight(20)plantFlower()end",\n    "CorrectAnswer": "2",\n    "Explanation": "To fix the existing algorithm, the robot needs to be positioned 20 cm inside the flower bed before planting the first flower."\n}'},
 {'MQCode': '2A17',
  'learning_objective': 'Describe choices made during program development using code comments, presentations, and demonstrations.',
  'topic': 'Function explanation',
  'question_str': '{\n    "Question": "Moonbeam is helping a classmate with a computer science coding assignment.  Moonbeam\'s classmate shows the following code:What code comment should Moonbeam advise their classmate to add to explain what functionality procA provides?",\n    "Answer1": "// procA takes 4 numbers as parameters and prints out the largest number",\n    "Answer2": "// procA takes 4 numbers as parameters and returns the smallest number",\n    "Answer3": "// procA takes 4 numbers as parameters and prints out the smallest number",\n    "Answer4": "// procA takes 4 numbers as parameters and returns the largest number",\n    "CorrectAnswer": "2",\n    "Explanation": "ProcB takes two numbers as parameters and returns the smaller one.   By calling ProcB several times, ProcA can take four numbers as parameters and return the smallest one.   Because the code in ProcA and ProcB returns values rather than output them to the screen, the results of ProcA and ProcB are not printed to the screen."\n}'},
 {'MQCode': '2C01',
  'learning_objective': 'Describe how internal and external parts of computing devices function to form a system.',
  'topic': 'Computer Input and Output',
  'question_str': '{\n    "Question": "Justice is working on a school lab computer.  Justice moves the computer mouse - \\u00a0\\u00a0\\u00a0(a)\\u00a0\\u00a0\\u00a0 -  which sends electronic signals that are interpreted by the computer operating system - \\u00a0\\u00a0\\u00a0(b)\\u00a0\\u00a0\\u00a0.   The operating system uses these electronic signals to move the cursor on the monitor to the OK button.Which of the following provides the best answers for (a) and (b)?",\n    "Answer1": "(a) = software, (b) = hardware",\n    "Answer2": "(a) = software, (b) = software",\n    "Answer3": "(a) = hardware, (b) = hardware",\n    "Answer4": "(a) = hardware, (b) = software",\n    "CorrectAnswer": "4",\n    "Explanation": "A computer mouse is hardware as it has physical components.   A computer operating system (e.g., Microsoft Windows) is software as it provides instructions to a computer and does not have any specific physical components associated with it. "\n}'},
 {'MQCode': '2D06',
  'learning_objective': 'Organize and present collected data visually to highlight relationships and support a claim.',
  'topic': 'Ice Cream Satisfaction',
  'question_str': '{\n    "Question": "Denali\'s class is studying their school\'s perception of different types of ice cream.  The results of this study were as follows:Students polledStudents extremely satisfiedPercentage extremely satisfiedIce Cream A504590%Ice Cream B705680%Ice Cream C605084%Students in Denali\'s class prepared the following 5 charts.  Which of the following charts best highlights that Ice Cream A had the highest percentage of students that were extremely satisfied?",\n    "Answer1": "Chart A",\n    "Answer2": "Chart B",\n    "Answer3": "Chart C",\n    "Answer4": "Chart D",\n    "CorrectAnswer": "1",\n    "Explanation": "Chart B shows the number of extremely satisfied students \\ufffd not the percentage satisfied.  Chart C, the pie chart, is inappropriate since it implies that the percentages shown have the same denominator.   Chart D, the line chart, is inappropriate because it implies that there is a linear relationship between Ice Cream A, B, and C."\n}'},
 {'MQCode': '2I18',
  'learning_objective': 'Discuss computing technologies that have changed the world, and express how those technologies influence, and are influenced by, cultural practices.',
  'topic': 'Cultural Misunderstanding',
  'question_str': '{\n    "Question": "Lightning is very proud to be selected as their school representative for a new social media website for student leaders in high schools worldwide. The student leaders will discuss issues affecting teens worldwide.  Lightning comes across a post by Amani - also a new representative - from a school in the Middle East discussing surgical masks and hajibs.  In response, Lightning sends a thumbs-up emoji \\ud83d\\udc4d with no further comment to show their approval. Amani replies with the comment \\"Why are you so rude?\\" followed by three different sad face emojis.Which of the following best explains why Amani did not view Lightning\'s comment as encouraging?",\n    "Answer1": "In some portions of the Middle East, the thumbs-up gesture is considered an offensive insult.    However, in most other countries, it is a sign of approval.  The website was developed by a US company and did not consider this international issue.",\n    "Answer2": "All Americans - especially teenagers - are rude",\n    "Answer3": "Amani had planned to send three smiling face emojis but clicked on three different sad face emojis instead.",\n    "Answer4": "In the Middle East, a frowning face emoji really means that one is happy.   However, in most other countries, it is a sign of sadness. ",\n    "CorrectAnswer": "1",\n    "Explanation": "There are many differences in how emoji use internationally and even within the same country.   When developing a software product, one cannot assume that the colors, gestures, emojis, etc have the same meaning as they do in the U.S.  When communicating internationally, be careful in use of emojis and consider using language that is difficult to misinterpret."\n}'},
 {'MQCode': '2I19',
  'learning_objective': 'Brainstorm ways to improve the accessibility and usability of technology products for the diverse needs and wants of users.',
  'topic': 'Online menu accessibility',
  'question_str': '{\n    "Question": "Cresent is creating an online menu for residents of a retirement community to order their meals from the community cafeteria.   For a typical meal, four main selections change daily.   In addition, the cafeteria offers 30-40 other items including main dishes, side orders, drinks, and desserts for each meal that change infrequently.  Many of the members of the retirement community have poor eyesight and other health issues associated with old age.Which of the following would NOT improve the accessibility and usability of the online menu when a resident uses it on a tablet computer?",\n    "Answer1": "Listing all 40 menu items alphabetically on a single screen",\n    "Answer2": "A feature that will suggest items to order based on a residents\' previous orders and stated preferences",\n    "Answer3": "A color scheme that is primarily large black text on a white background with a few color accents",\n    "Answer4": "An interface with large buttons that takes the resident step by step to order the main dish, then side orders, then drinks, and finally dessert.",\n    "CorrectAnswer": "1",\n    "Explanation": "Listing all 40 menu items alphabetically will force much text onto a small screen making it difficult for residents with poor vision to read and to find what the main dishes are for that meal."\n}'},
 {'MQCode': '2N04',
  'learning_objective': 'Model how information is broken down into smaller pieces, transmitted as packets through multiple devices over networks and the Internet, and reassembled at the destination.',
  'topic': 'Video streaming process',
  'question_str': '{\n    "Question": "Starlight is streaming videos on their smartphone from an education video website.  The video is 63 minutes long in high definition.   Which of the following best explains how the video is sent from the education video website to Starlight\'s smartphone after a connection is made between the video website and the smartphone?",\n    "Answer1": "A dedicated pathway is created between a server on the website and the smartphone.  The video file is first downloaded in its entirety to the smartphone before the video begins.",\n    "Answer2": "A dedicated pathway is created between a server on the website and the smartphone.  The video file is broken up into thousands of packets, sent over the dedicated pathway, and displayed as packets arrive. ",\n    "Answer3": "The video file is broken up into thousands of packets, sent over the Internet via multiple pathways, and displayed on the smartphone as packets arrive.",\n    "Answer4": "The video file is broken up into thousands of packets, sent over the Internet via multiple pathways, and reassembled in its entirety on the smartphone.   After the video file is entirely reassembled, the video starts on the smartphone ",\n    "CorrectAnswer": "3",\n    "Explanation": "TCP/IP stands for Transmission Control Protocol/Internet Protocol.   Once the video server knows the IP address to steam the video to, it uses TCP to break the file up into multiple packets, attaches the IP address to each packet, and then sends out over the Intent.   The Internet takes each packet and routes it to the smartphone; however, each packet could take a different route depending on the network conditions.   The packets are reassembled in order by TCP as they arrive and are buffered on the smartphone.   If a packet fails to arrive, it is re-requested."\n}'},
 {'MQCode': '2N05',
  'learning_objective': 'Discuss real-world cybersecurity problems and how personal information can be protected.',
  'topic': 'Phishing attempt',
  'question_str': '{\n    "Question": "Sandy receives an e-mail from Google Security indicating that their Google account has been disabled.   The e-mail instructs Sandy to click on a button on the e-mail and input their username and password to recover their account and change their password.   What should Sandy do?",\n    "Answer1": "Sandy\'s Google account is blocked.  Sandy should follow the instructions in the e-mail by clicking the button on the e-mail and entering their e-mail and password on the page to recover their account.",\n    "Answer2": "This is likely a phishing attempt to get Sandy\'s password.    Sandy should check the e-mail address used to send the mail, and if suspicious, mark the e-mail as phishing or as junk. ",\n    "Answer3": "Sandy\'s Google account is blocked.  Sandy should reply to the e-mail and include their username and password to request Google Security to restore access to the account.",\n    "Answer4": "Sandy should forward the e-mail to their parents to ask for advice.",\n    "CorrectAnswer": "2",\n    "Explanation": "This is likely a phishing attempt. Reputable services will NOT send e-mail informing customers that their service is blocked.   Never click on a link on such a mail or send your password in plaintext over the internet.   If one is having problems with a service, visit the service directly \\ufffd without using the link \\ufffd and try logging in."\n}'},
 {'MQCode': '3A10',
  'learning_objective': 'Use flowcharts and/or pseudocode to address complex problems as algorithms.',
  'topic': 'Robot maze navigation',
  'question_str': '{\n    "Question": "Trekkie is programming a robot to navigate through a maze by using the \\"right-hand rule\\":If you can walk forward, then move forward. If you can\'t,  then turn right.  Repeat.A flowchart for Trekkie program looks as follows:Which of the following code snippets best represents the algorithm in the flowchart?",\n    "Answer1": "if canMoveForward()moveForward()elseturnRight()end",\n    "Answer2": "repeat while endOfMaze()if canMoveForward()turnRight()elsemoveforward()endend",\n    "Answer3": "repeat until endOfMaze()if canMoveForward()moveForward()elseturnRight()endend",\n    "Answer4": "repeat until endOfMaze()moveForward()turnRight()end",\n    "CorrectAnswer": "2",\n    "Explanation": "The flowchart shows a conditional within a loop.  Because it can receive control back from the bottom of the flowchart, the End maze? diamond shape at the top represents a repeat while loop.  Because all its branches eventually flow back to End Maze, the Can Move Forward diamond shape represents an if statement that is contained within the repeat until loop."\n}'},
 {'MQCode': '3A11',
  'learning_objective': 'Create clearly named variables that represent different data types and perform operations on their values.',
  'topic': 'Animal counting program',
  'question_str': '{\n    "Question": "Lava is writing a program to count the number of each type of animal on various farms based on a picture of each animal.   A code snippet from the initial iteration of Lava\'s program looks as follows:The use of the variable names a, b, and c make this code snippet difficult to understand.   Which of the following replacements for the variable names a, b, and c would make this code the easiest to understand?",\n    "Answer1": "a: horse, b: isHorse, c: horses",\n    "Answer2": "a: textAnimal, b: isHorse, c: countAnimal",\n    "Answer3": "a: countAnimal, b: isAnimal, c: textAnimal",\n    "Answer4": "a: textStudent, b:isStudent, c:countStudent",\n    "CorrectAnswer": "2",\n    "Explanation": "Variable b holds a yes/no value of whether a picture is of a horse.  Variable a and c could be used for any type of animal.  Variable a is a text string with the name of the animal; variable c is the count of the animal type, and variable c contains the count of horses - 1 if the picture was a horse or 0 otherwise."\n}'},
 {'MQCode': '3A12',
  'learning_objective': 'Design and iteratively develop programs that combine control structures, including nested loops and compound conditionals.',
  'topic': 'Checkerboard design',
  'question_str': '{\n    "Question": "Yoyo is writing a procedure that creates checkerboard designs of various sizes with x\\ufffds and o\\ufffds.  An early version of the procedure is promising, but not quite correct. Some of the designs this early version creates are as follows:Example A (i=10)Example B (i=8)Example C (i=6)xoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoxoWhich of the following most closely resembles the early version of Yoyo\'s checkerboard code that produced the above results?",\n    "Answer1": "action board(integer i)if i = 10output \\"xoxoxoxoxo\\"elseif i = 8output \\"xoxoxoxo\\"elseoutput \\"xoxoxo\\"endend",\n    "Answer2": "action board(integer i)text pattern = \\"\\"repeat i timesrepeat i timespattern = pattern + \\"xo\\"endendoutput patternend",\n    "Answer3": "action board(integer i)repeat i timestext pattern = \\"\\"repeat i / 2 timespattern = pattern + \\"xo\\"endoutput patternendend",\n    "Answer4": "action board(integer i)repeat 8 timesoutput \\"xoxoxoxo\\"endend",\n    "CorrectAnswer": "3",\n    "Explanation": "Creating these designs requires one loop nested within another.   The outer loop outputs each line in the design. The inner loop creates each line with half as many \\"xo\\" repeated as there are number of lines.   If i is an even number, then the number of characters on each line equals the number of lines."\n}'},
 {'MQCode': '3A13',
  'learning_objective': 'Decompose problems and subproblems into parts to facilitate the design, implementation, and review of programs.',
  'topic': 'World Cup song',
  'question_str': '{\n    "Question": "Goalie loves the World Cup and their favorite player Lionel Messi.   Goalie is trying to write a computer program to generate a song for the 2022 World Cup which lasts 28 days.   A partial early version of the program is below:What output will be produced by a call to the procedure Day3()?",\n    "Answer1": "Winning goal by Lionel MessiWinning goal by Lionel MessiTwo corner kicksThree throw-ins",\n    "Answer2": "Three throw-insTwo corner kicksWinning goal by Lionel MessiWinning goal by Lionel Messi",\n    "Answer3": "Three throw-insTwo corner kicksWinning goal by Lionel Messi",\n    "Answer4": "Winning goal by Lionel MessiTwo corner kicksThree throw-ins",\n    "CorrectAnswer": "1",\n    "Explanation": "Understand control flow to break down the output by each line in the Day3 procedure:Line #StatementOutputLine 2Day1()//Winning goal by Lionel MessiLine 3Day2()//Winning goal by Lionel Messi//Two corner kicksLine 4output \\"Three throw-ins\\"//Three throw-ins"\n}'},
 {'MQCode': '3A14',
  'learning_objective': 'Create procedures with parameters to organize code and make it easier to reuse.',
  'topic': 'Creating repeated characters',
  'question_str': '{\n    "Question": "Quartz is writing a program to produce art by printing out characters.    Some simple examples of the art that Quartz would like to create are below:Example 1Example 2Example 3AbbCCCdddd---****-----**oo**---**oooo**-**oooooo**11333355555577777777Quartz realizes that to create this type of simple artwork, they must write a procedure to create strings with the same character repeated multiple times.  Which of the following code snippets best represents the procedure that Quartz should write:",\n    "Answer1": "action RepeatChar(text ch, integer num) returns texttext str = \\"\\"repeat num timesstr = str + chendreturn strend",\n    "Answer2": "action RepeatChar()text str = \\"\\"repeat num timesstr = str + chendreturn strend",\n    "Answer3": "action RepeatChar(text ch, integer num) returns textreturn chend",\n    "Answer4": "action RepeatChar(text ch, integer num) returns textrepeat num timesstr = str + chendend",\n    "CorrectAnswer": "1",\n    "Explanation": "A procedure to create strings with the same character multiple times must: 1) have parameters for the character to be repeated and the number of repetitions, and 2) return a text string with the same character repeated multiple times usually by using a loop. Answer A is the only answer that fulfill these two requirements"\n}'},
 {'MQCode': '3A17',
  'learning_objective': 'Systematically test and refine programs using a range of test cases.',
  'topic': 'Temperature classification',
  'question_str': '{\n    "Question": "Surfer is coding a procedure that will take a temperature in Celsius as an parameter and print out whether water will become ice ( <= 0\\ufffd ), boil ( >= 100\\ufffd) or remain water.   The code for Surfer\'s procedure is as follows:For which of the following calls to waterState() will Surfer\'s procedure output misleading results?",\n    "Answer1": "waterState(120)",\n    "Answer2": "waterState(50)",\n    "Answer3": "waterState(0)",\n    "Answer4": "waterState(-5)",\n    "CorrectAnswer": "1",\n    "Explanation": "At a temperature of 120 degrees Celsius, waterState() will output both the phrase \\"Water will boil\\" AND the phrase \\"Water will not freeze or boil\\".   waterState() should use a single if \\ufffd elseif \\ufffd else conditional for the best results. "\n}'},
 {'MQCode': '3D07',
  'learning_objective': 'Represent data using multiple encoding schemes.',
  'topic': 'Binary to hexadecimal conversion',
  'question_str': '{\n    "Question": "Ocean has learned that the first letter in their name \'O\' has the same binary encoding (01001111) as the number 79.   Because remembering the binary digits of \'O\' is difficult, Ocean wants to remember the hexadecimal encoding.What is the hexadecimal encoding for the letter \'O\'?",\n    "Answer1": "\\u00a0\\u00a079\\u00a0\\u00a0",\n    "Answer2": "\\u00a0\\u00a04F\\u00a0\\u00a0",\n    "Answer3": "\\u00a0\\u00a097\\u00a0\\u00a0",\n    "Answer4": "\\u00a0\\u00a0117\\u00a0\\u00a0",\n    "CorrectAnswer": "2",\n    "Explanation": "Hexadecimal is base 16 with digits 0 -  9 and A - F for 11 through 15.   One can convert from binary to hexadecimal by taking the binary digits four at a time.  \'1111\' is \'F\' and \'0100\' is \'4\'.   Thus \'4F\' is the hexadecimal representation of the letter \'O\'."\n}'},
 {'MQCode': '3D09',
  'learning_objective': 'Refine computational models based on the data they have generated.',
  'topic': 'Popsicle temperature prediction',
  'question_str': '{\n    "Question": "Palm sells popsicles at track meets and wants to predict the number of popsicles sold based on the temperature in Fahrenheit at the start of the meet.  The chart below shows how many popsicles Palm sold at each meet last year:Which of the following code snippets provides the best prediction of the number of popsicles sold depending on the temperature?",\n    "Answer1": "if temp >= 85output \\"50 popsicles sold\\"elseif temp <= 45output \\"0 popsicles sold\\"elseoutput 1.25 * (temp - 45) + \\" popsicles sold\\"end",\n    "Answer2": "output (temp - 45) + \\" popsicles sold\\"",\n    "Answer3": "if temp >= 85output \\"50 popsicles sold\\"elseif temp <= 45output \\"0 popsicles sold\\"elseoutput temp + \\" popsicles sold\\"end",\n    "Answer4": "if temp < 85output \\"50 popsicles sold\\"else temp > 45output \\"0 popsicles sold\\"end",\n    "CorrectAnswer": "1",\n    "Explanation": "There are three conditions that the prediction must cover: 1) if the temperature is greater than 85 degrees, then all 50 bottles are sold; 2) if the temperature is less than 45 degrees, then 0 bottles are sold 3) if the temperature is between 45 and 85 degrees, there is a linear relationship between the number of popsicles sold and the temperature.   Only Answer A correctly handles all 3 conditions. "\n}'},
 {'MQCode': '3I20',
  'learning_objective': "Compare tradeoffs associated with computing technologies that affect people's everyday activities and career options.",
  'topic': 'Future police officers',
  'question_str': '{\n    "Question": "\\"When I grow up, I want to be a police officer and catch criminals who steal others\' money,\\ufffd said eight-year-old Beach.  \\"So, I don\\ufffdt need to learn computer science in school.\\"\\"You are wrong,\\" replied thirteen-year-old Lagos.   \\"If you want to be a police officer, you must still learn computer science because ...\\"Which of the following would Lagos MOST LIKELY give as a reason why future police officers should learn computer science?",\n    "Answer1": "Computer programming is a significant portion of the exam to be a police officer.",\n    "Answer2": "Police officers who graduate with computer science degrees make 3 times as much starting salary as those who don\\ufffdt.",\n    "Answer3": "Stealing money using computing technology is easier and less risky for criminals worldwide than other methods.  Cybercrime is rapidly increasing.",\n    "Answer4": "Police departments all have the most advanced computing technology available as shown on television.",\n    "CorrectAnswer": "3",\n    "Explanation": "Because cybercrime is rapidly increasing, so is the need for law enforcement to know both cybersecurity and computer forensics.  Because so much data about our lives is tracked digitally, law enforcement also needs computer science skills to search through this data and efficiently and effectively investigate criminal activity."\n}'},
 {'MQCode': '3I23',
  'learning_objective': 'Describe tradeoffs between allowing information to be public and keeping information private and secure.',
  'topic': 'Code confidentiality',
  'question_str': '{\n    "Question": "For a high school project, Chengdu has developed an app for a game like the old Windows game Minesweeper and allows friends to challenge and compare scores with each other online.  Chengdu\'s teacher comments that the work on the app is \\"fantastic.\\"   Chengdu wonders whether they should publish their code online on GitHub or keep it secret.Which of the following is a reason why Chengdu should NOT publish their code online without futher modification?",\n    "Answer1": "The code may contain certain passwords or other configuration information that Chengdu needs to keep secret.",\n    "Answer2": "It will be harder for Chengdu to share the code with others to collaborate in the future.",\n    "Answer3": "Someone may copy the code and resell it for a billion dollars.",\n    "Answer4": "Potential employers will see the code and know that Chengdu  is a bad programmer.",\n    "CorrectAnswer": "1",\n    "Explanation": "Especially because the app shares data online, the code likely contains passwords and other configuration information that needs to be kept secret.   Release of this secret information may result in  Chengdu unexpectedly having to pay for online computing resources or having their game lose online access.  Fortunately, online repositories such as Github do provide ways to keep confidential configuration information private.  Other parts of the code provide functionality that is similar to what many others have done so publishing the code online is beneficial. "\n}'},
 {'MQCode': '3N05',
  'learning_objective': 'Explain how physical and digital security measures protect electronic information.',
  'topic': 'Two-factor authentication',
  'question_str': '{\n    "Question": "After Berkeley visits their favorite social media website and inputs their username and password, they receive a text on their smartphone with a code that they must input into the site to verify their identity.   While Berkeley finds this process somewhat time-consuming, they appreciate that this method provides both digital and physical methods to verify their identity. What is this procedure of requiring both a password and a code sent via text or e-mail is also known as?",\n    "Answer1": "biometric authentication",\n    "Answer2": "double identification",\n    "Answer3": "two-factor authentication",\n    "Answer4": "dual authorization",\n    "CorrectAnswer": "3",\n    "Explanation": "Two-factor authentication.   Authentication is the process of proving that one is how one identifies themself.   Since two methods of authentication are used here, the term is two-factor authentication. "\n}'},
 {'MQCode': '3N06',
  'learning_objective': 'Apply multiple methods of encryption to model the secure transmission of information.',
  'topic': 'Caesar cipher encryption',
  'question_str': '{\n    "Question": "Nashville is sending a message to Memphis using Caesar cipher encryption to keep it a secret from 8-year-old Raleigh. The original message is:WE WILL EAT PIZZA TONIGHTThe encrypted message is:DL DPSS LHA WPGGH AVUPNOAWhat is the shift value that Nashville used for the Caesar cipher to encrypt this message?",\n    "Answer1": "3",\n    "Answer2": "5",\n    "Answer3": "7",\n    "Answer4": "10",\n    "CorrectAnswer": "3",\n    "Explanation": "The Caesar cipher is one of the simplest encryption techniques in which each letter in a message is replaced by a letter that is some fixed number of positions (shift value) down in the alphabet.   With a shift value of 7, each letter of the alphabet is replaced as follows:A B C D E F G H I J K L M N O P Q R S T U V W X Y ZH I J K L M N O P Q R S T U V W X Y Z A B C D E F GBecause the Caesar cipher is easy to decrypt even without the shift value, it is not used in much in real-life."\n}'},
 {'MQCode': '4A14',
  'learning_objective': 'Use lists to simplify solutions, generalizing computational problems instead of repeatedly using simple variables.',
  'topic': 'Summing list of numbers',
  'question_str': '{\n    "Question": "Courage knows how to write code to find the sum of two numbers, three numbers, or even four numbers, but wonders how to write code that can find the sum of one hundred, one thousand, or even a million numbers. Which one of the following best represents a procedure that will return the sum of a list of numbers that can be any length?",\n    "Answer1": "action AddList(List myList) returns numbernumber sum = 0.0integer position = 0repeat while position > myList:GetSize()sum = sum + myList:GetSize()position = position + 1endreturn sumend",\n    "Answer2": "action AddList(List myList) returns numbernumber sum = 0.0integer position = 0repeat while position > myList:GetSize()sum = sum + myList:Get(position)endreturn sumend",\n    "Answer3": "action AddList(List myList) returns numbernumber sum = 0.0integer position = 0repeat while position > myList:GetSize()sum = sum + myList:Get(position)position = position + 1endreturn sumend",\n    "Answer4": "action AddList(List myList) returns numbernumber sum = 0.0integer position = 0repeat while position < myList:GetSize()sum = sum + myList:Get(position)position = position + 1endreturn sumend",\n    "CorrectAnswer": "4",\n    "Explanation": "The correct procedure initializes the variables sum = 0 and position = 0.  In most major computer languages, the first element in a list starts at position 0 and the last element is at GetSize() - 1.  Within the loop, the number at each list item is added to the running sum, and the variable position is incremented.   After all numbers in the list have been added to the running sum, the running sum is returned. "\n}'},
 {'MQCode': '4C02',
  'learning_objective': 'Compare levels of abstraction and interactions between application software, system software, and hardware layers.',
  'topic': 'Computer file',
  'question_str': '{\n    "Question": "Following their teacher\\ufffds instructions, Brilliant opens the word processing document that they started yesterday.  The operating system on Brilliant\'s computer receives the request to open the document and delivers it to the program as a single stream of bytes even though it may exist in several fragments in different locations on the computer hard drive.What abstraction provides the most value in the above scenario?",\n    "Answer1": "document",\n    "Answer2": "teacher",\n    "Answer3": "fragments",\n    "Answer4": "bytes",\n    "CorrectAnswer": "1",\n    "Explanation": "Document.  An abstraction in computer science is generalizing the details of an object or system to focus attention on details of greater importance.   The operating system provides the abstraction of a document to the word processing program so that the word processing program does not need to figure out how to retrieve a sequential stream of bytes from several fragmented locations on a network."\n}'},
 {'MQCode': '4D09',
  'learning_objective': 'Translate between different bit representations of real-world phenomena, such as characters, numbers, and images.',
  'topic': 'Unicode and HTML',
  'question_str': '{\n    "Question": "While making a Valentine\'s Day website for a school project, Diamond learns that a favorite emoji, the Red Heart, has a Unicode code point U+2764.  Because Unicode code points are expressed in hexadecimal, that means that Red Heart emoji is represented in smartphone texts by the following bits: 0010 0111 0110 0100.   However, Diamond is curious why they must use the HTML entity code &#10084; rather than &#2764;  to insert a Red Heart emoji onto their website.Which of the following best explains why &#10084; is used to insert the Red Heart emoji on websites?",\n    "Answer1": "Unicode code points and HTML escape codes were established independently.  The mapping between the two is stored in an online database. ",\n    "Answer2": "Each browser maker has its mapping between Unicode and HTML escape codes and stores a unique mapping in its logic.",\n    "Answer3": "The HTML entity code of an emoji uses the decimal equivalent of the Unicode code point.",\n    "Answer4": "A digital visual representation of each emoji is stored on each computing device.",\n    "CorrectAnswer": "3",\n    "Explanation": "The HTML entity code of an emoji uses the decimal equivalent of the Unicode code point.  This enables browsers to quickly map the HTML escape code to the digital visual representation of the emoji stored on the computing device according to the Unicode code point.  Hexadecimal 2764 = 2 * 4096 + 7 * 256 + 6 * 16 + 4 = 10084."\n}'},
 {'MQCode': '4I25',
  'learning_objective': 'Test and refine computational artifacts to reduce bias and equity deficits.',
  'topic': 'Pet photo AI',
  'question_str': '{\n    "Question": "Sunny is furious.   An AI on the Internet that rates the health of a pet based on a photo rated Sunny\'s cat Bengal as \\"very overweight and unhealthy\\" although Sunny knows that Bengal is of average weight and is very healthy from their last visit to the veterinarian.   Sunny finds an article on the Internet that mentions that while the pet photo health AI is accurate with dogs, it often miscalculates the weight of cats several pounds higher than actual. Which of the following should Sunny recommend to the developer of the pet photo health AI to fix their cat problem?",\n    "Answer1": "The developer should check their calculation of volume: Volume = height * width * depth.",\n    "Answer2": "The developer should make their UI more accessible.",\n    "Answer3": "The developer should test and refine their AI with many more cats.",\n    "Answer4": "The developer should refund all users with cats $10.   Users did not need to pay for the initial diagnosis.",\n    "CorrectAnswer": "3",\n    "Explanation": "Errors in AI is often caused by biased data or lack of data.  Testing and refining their AI with more photos of cats with the actual cat weight will improve the accuracy of the AI with cats. "\n}'},
 {'MQCode': '4I26',
  'learning_objective': 'Demonstrate ways a given algorithm applies to problems across disciplines.',
  'topic': 'Sorting and Searching',
  'question_str': '{\n    "Question": "Arora has developed an algorithm to sort a list of objects in alphabetical order by a field and another algorithm to search whether a matching item exists in that list.   Two applications that Arora has developed using these algorithms are 1) converting an Internet domain name to an IP address and 2) using a product id to update store inventory when a product is purchased.Which of the following is another application that Arora could use their algorithms for?",\n    "Answer1": "Providing an AI to play tic-tac-toe",\n    "Answer2": "Predicting the probability that it will rain tomorrow",\n    "Answer3": "Calculating the average daily bill of customers at a restaurant",\n    "Answer4": "Spell-checking a document",\n    "CorrectAnswer": "4",\n    "Explanation": "Spell-checking a document requires first sorting the list of valid words.   If the binary search algorithm cannot find the word in the list, then it is flagged as incorrect."\n}'},
 {'MQCode': '4I28',
  'learning_objective': 'Explain the beneficial and harmful effects that intellectual property laws can have on innovation.',
  'topic': 'Copyright laws and app development',
  'question_str': '{\n    "Question": "Galaxy has an exciting idea for a new app that they think can make millions of dollars in sales over the next three years.   After an exhaustive search on the Internet, Galaxy finds a competitor app that does about 75 percent of Galaxy\'s idea but is slow and has an ugly and difficult-to-use user interface.   This competitor app displays a copyright notice when it first launches.Which one of the following is a reason why Galaxy may wish that copyright laws were different in the United States?",\n    "Answer1": "By placing a copyright on their app once it is completed, Galaxy can protect themselves from having their ideas stolen or copied.",\n    "Answer2": "Copyright laws promote the free flow of information and ideas in society.",\n    "Answer3": "The copyright on the competitor app will make it difficult for Galaxy to use it as a starting point for their app.",\n    "Answer4": "By placing a copyright of their app once it is completed, Galaxy will have to pay 30% of all its revenue to the U.S. government.",\n    "CorrectAnswer": "3",\n    "Explanation": "Copyright is a type of intellectual property that protects original works of authorship including computer programs as soon as an author fixes the work in a tangible form of expression.   Although Galaxy may develop their app independently of its competitor, they may not use or copy the code of the competitor app without permission.  Once Galaxy completes their app, their app is protected by copyright as soon as it is published.   Further, Galaxy can get a registered copyright for their app from the U.S. government for a nominal fee (usually less than $100)"\n}'},
 {'MQCode': '4I30',
  'learning_objective': 'Evaluate the social and economic implications of privacy in the context of safety, law, or ethics.',
  'topic': 'Privacy concerns',
  'question_str': '{\n    "Question": "Sixteen-year-old Comet has developed an application that will recommend music playlists for high school students based on their school, currently popular music, the time of day, their location, their current activity, and previous music selections.   Students using the app must provide an e-mail and mobile phone number to use the application.  Comet\'s good friend Temperance is amazed by how well Comet\'s application works but is concerned about the privacy provided by Comet\'s application.Which of the following is likely one of Temperance\'s privacy concerns?",\n    "Answer1": "Comet will need to do much more development work and perhaps invest in some security software to implement privacy safeguards.",\n    "Answer2": "If students\' private information in Comet\'s application is leaked or misused, students will forgive Comet because the application is so popular.",\n    "Answer3": "Because the application is for high school students, Comet does not need to worry about children under 13 using the application and the Children\'s Online Privacy Protection Act.",\n    "Answer4": "The lack of privacy safeguards in Comet\'s current application makes it harder to sell personalized ads.",\n    "CorrectAnswer": "1",\n    "Explanation": "Properly developing an application - and maintaining it - to address privacy and security issues requires considerable time and effort beyond just developing the basic functionality provided by the application.  Privacy in computer software has many social, economic, legal, ethical, and safety implications."\n}'},
 {'MQCode': '4N04',
  'learning_objective': 'Evaluate the scalability and reliability of networks, by describing the relationship between routers, switches, servers, topology, and addressing.',
  'topic': 'Network Router',
  'question_str': '{\n    "Question": "Golden is installing a wireless network for their family.  Golden\'s family already owns a network switch which connects several wireless devices and two wired servers in their home to the same network.   But Golden\'s family needs an additional device to connect their home network to the Internet.   What type of device should Golden inform their family to purchase?",\n    "Answer1": "Network address",\n    "Answer2": "Network server",\n    "Answer3": "Network hub",\n    "Answer4": "Network router",\n    "CorrectAnswer": "4",\n    "Explanation": "A network router is a device that connects one network such as a home local area network to another such as the Internet.  Homes often have devices that combine both network routers and switches, but the two can be purchased separately."\n}'},
 {'MQCode': '4N05',
  'learning_objective': 'Give examples to illustrate how sensitive data can be affected by malware and other attacks.',
  'topic': 'Malware and Society',
  'question_str': '{\n    "Question": "Cipher is writing a report on malware and its effect on modern society and comes across the following articles.   Which of these are likely to be useful to Cipher in writing their report?",\n    "Answer1": "Major storms damage deep water fiber optic cables",\n    "Answer2": "Botnet brings down major bank web site through denial-of-service attack",\n    "Answer3": "Healthcare providers lost over $500 million in ransomware during 2021",\n    "Answer4": "Both B and C",\n    "CorrectAnswer": "4",\n    "Explanation": "Malware - short for malicious software - is any software intentionally designed to cause disruption to a computing system or compromise its security or privacy.  Ransomware is a type of malware that threatens to publish the victim\'s data or permanently block access to it unless a ransom is paid.   A botnet is a group of Internet-connected devices infected by malware that are under the control of a single attacking party."\n}'},
 {'MQCode': '4N07',
  'learning_objective': 'Compare various security measures, considering tradeoffs between the usability and security of a computing system.',
  'topic': 'Improving network security',
  'question_str': '{\n    "Question": "Enthusiasm was shocked when they heard their principal announce over the intercom: \\"Earlier this week our school was hacked.  We had to pay a ransom to restore school data and must reimage all school computers to remove malware that the hackers installed... To prevent this from happening again, we are implementing the following changes that may be inconvenient but will improve school network security.   The first change is that all passwords must be at least 12 characters and sufficiently complex...\\"Which of the following is another change that the principal is likely to add to this announcement?",\n    "Answer1": "Shutting off all access to district computing resources outside of school hours.",\n    "Answer2": "Use of two-factor authentication when logging in from computers outside of the school network.",\n    "Answer3": "Blocking of traffic to all URLs except for 10 websites approved by the school district.",\n    "Answer4": "Locking away the servers storing student grades in a locked room in a secure facility with military guards.",\n    "CorrectAnswer": "2",\n    "Explanation": "Two-factor authentication such as typing in a code sent to a mobile phone owned by the user is a common way to provide a more secure computing environment.   While these other methods suggested would provide more security, they would greatly limit the usefulness of the computing environment beyond the security benefit they would provide."\n}'}
]


Add coding examples.

In [179]:
with open("codeexamples.txt", 'r') as file:
    # Read the content of the file
    content = file.read()

coding_content = re.split(r"\*\* (\w{4}) \*\*", content)[1:]

mqCodes = [mqcode for i, mqcode in enumerate(coding_content) if i % 2 == 0]
coding_snippets = [snippet for i, snippet in enumerate(coding_content) if i % 2 == 1]

idxs = []
for i, ex in enumerate(examples):
    if ex['MQCode'] in mqCodes:
        idxs.append(i)
        
coding_q_exs = []
for idx, code_snippet in zip(idxs, coding_snippets):
    updated_q = examples[idx]
    q_json = json.loads(updated_q['question_str'])
    q_json['code_snippet'] = code_snippet[2:-2]
    updated_q['question_str'] = json.dumps(q_json, indent=4)
    coding_q_exs.append(updated_q)

In [187]:
### Full list of coding examples. (result of previous cell)
coding_q_exs = [
 {'MQCode': '1A14',
  'learning_objective': 'Debug (identify and fix) errors in an algorithm or program that includes sequences and simple loops.',
  'topic': 'Fixing temperature program',
  'question_str': '{\n    "Question": "Jude is writing a program to find the coldest temperature in the previous year.  Jude has a list of the low temperature for each day that looks as follows:23.521.743.0\\u2026\\u202635.9The original program runs but gives the wrong answer:What change should Jude make to fix the program and output the coldest temperature?",\n    "Answer1": "Modify line 4: if dayCold > maxCold",\n    "Answer2": "Modify line 4: if dayCold = maxCold",\n    "Answer3": "Modify line 5: dayCold = maxCold",\n    "Answer4": "Modify line 1: number maxCold = 1000.0",\n    "CorrectAnswer": "4",\n    "Explanation": "Because maxCold is set so low in line 1, line 4 is never true and maxCold will always be -1000.0.   By modifying line 1 to maxCold = 1000.0 - higher than any expected value, the first value read from the list will become the initial maxCold and the program will work as expected.",\n    "code_snippet": "number maxCold = -1000.0  //never colder than -1000.00 degrees\\nwhile hasNextNumber()\\n    number dayCold = getNextNumber()\\n    if dayCold < maxCold\\n        maxCold = dayCold\\n    end\\nend\\noutput maxCold"\n}'},
 {'MQCode': '1I18',
  'learning_objective': 'Keep login information private, and log off of devices appropriately.',
  'topic': 'School email incident',
  'question_str': '{\n    "Question": "Red is angry.  Yesterday after class, someone e-mailed the following message to Sky using Red\\ufffds school account:Red does not have a crush on Sky and does not remember sending the message.  Which of the following is the most likely explanation for the message?",\n    "Answer1": "Red has a crush on Sky and sent the e-mail to Sky while sleepwalking the previous night.",\n    "Answer2": "Red changed the password for their school account yesterday.",\n    "Answer3": "Red often forgets to log off from the computer at the end of class.   Someone noticed yesterday and sent the mail using Red\'s account.",\n    "Answer4": "Red clicked on an e-mail with the subject line \\"Nigerian prince wants money\\" but realized it was a joke sent by their best friend Moss. ",\n    "CorrectAnswer": "3",\n    "Explanation": "Keep your login information private, and remember to log off of devices appropriately.",\n    "code_snippet": "From: Red\\nTo: Sky\\nSubject: Crush\\n\\nI have a GIANT CRUSH on you!!!!\\nEver since 2nd grade"\n}'},
 {'MQCode': '2A10',
  'learning_objective': 'Create programs that include sequences, events, loops, and conditionals.',
  'topic': 'Variables and output',
  'question_str': '{\n    "Question": "To better understand how variables and programming work, Summit creates the following code:What does this code output on line 4?",\n    "Answer1": "\\u00a0\\u00a03\\u00a0\\u00a0",\n    "Answer2": "\\u00a0\\u00a05\\u00a0\\u00a0",\n    "Answer3": "\\u00a0\\u00a06\\u00a0\\u00a0",\n    "Answer4": "\\u00a0\\u00a08\\u00a0\\u00a0",\n    "CorrectAnswer": "4",\n    "Explanation": "Use code tracing to understand the value of each expression at each step.Line 1a=3Line 2a=3, b=3Line 3a=5, b=3Line 4a+b = 8",\n    "code_snippet": "text a = \\"horse\\"\\n// TODO: Write procedures to determine other animals\\nboolean b = isAnimalAHorse(animal)\\ninteger c = 0\\nif b\\n    c = c + 1\\nend\\noutput \\"Number of \\" + a + (s): \\" + c"\n}'},
 {'MQCode': '2A17',
  'learning_objective': 'Describe choices made during program development using code comments, presentations, and demonstrations.',
  'topic': 'Function explanation',
  'question_str': '{\n    "Question": "Moonbeam is helping a classmate with a computer science coding assignment.  Moonbeam\'s classmate shows the following code:What code comment should Moonbeam advise their classmate to add to explain what functionality procA provides?",\n    "Answer1": "// procA takes 4 numbers as parameters and prints out the largest number",\n    "Answer2": "// procA takes 4 numbers as parameters and returns the smallest number",\n    "Answer3": "// procA takes 4 numbers as parameters and prints out the smallest number",\n    "Answer4": "// procA takes 4 numbers as parameters and returns the largest number",\n    "CorrectAnswer": "2",\n    "Explanation": "ProcB takes two numbers as parameters and returns the smaller one.   By calling ProcB several times, ProcA can take four numbers as parameters and return the smallest one.   Because the code in ProcA and ProcB returns values rather than output them to the screen, the results of ProcA and ProcB are not printed to the screen.",\n    "code_snippet": "integer a = 3\\ninteger b = a\\na = a + 2\\noutput (a + b)"\n}'},
 {'MQCode': '3A11',
  'learning_objective': 'Create clearly named variables that represent different data types and perform operations on their values.',
  'topic': 'Animal counting program',
  'question_str': '{\n    "Question": "Lava is writing a program to count the number of each type of animal on various farms based on a picture of each animal.   A code snippet from the initial iteration of Lava\'s program looks as follows:The use of the variable names a, b, and c make this code snippet difficult to understand.   Which of the following replacements for the variable names a, b, and c would make this code the easiest to understand?",\n    "Answer1": "a: horse, b: isHorse, c: horses",\n    "Answer2": "a: textAnimal, b: isHorse, c: countAnimal",\n    "Answer3": "a: countAnimal, b: isAnimal, c: textAnimal",\n    "Answer4": "a: textStudent, b:isStudent, c:countStudent",\n    "CorrectAnswer": "2",\n    "Explanation": "Variable b holds a yes/no value of whether a picture is of a horse.  Variable a and c could be used for any type of animal.  Variable a is a text string with the name of the animal; variable c is the count of the animal type, and variable c contains the count of horses - 1 if the picture was a horse or 0 otherwise.",\n    "code_snippet": "action procA (number a, number b, number c, number d) returns number\\n    return procB(procB(a,b), procB(c,d))\\nend\\n\\naction procB (number a, number b) returns number\\n    if a < b\\n        return a\\n    else\\n        return b\\n    end\\nend  "\n}'},
 {'MQCode': '3A13',
  'learning_objective': 'Decompose problems and subproblems into parts to facilitate the design, implementation, and review of programs.',
  'topic': 'World Cup song',
  'question_str': '{\n    "Question": "Goalie loves the World Cup and their favorite player Lionel Messi.   Goalie is trying to write a computer program to generate a song for the 2022 World Cup which lasts 28 days.   A partial early version of the program is below:What output will be produced by a call to the procedure Day3()?",\n    "Answer1": "Winning goal by Lionel MessiWinning goal by Lionel MessiTwo corner kicksThree throw-ins",\n    "Answer2": "Three throw-insTwo corner kicksWinning goal by Lionel MessiWinning goal by Lionel Messi",\n    "Answer3": "Three throw-insTwo corner kicksWinning goal by Lionel Messi",\n    "Answer4": "Winning goal by Lionel MessiTwo corner kicksThree throw-ins",\n    "CorrectAnswer": "1",\n    "Explanation": "Understand control flow to break down the output by each line in the Day3 procedure:Line #StatementOutputLine 2Day1()//Winning goal by Lionel MessiLine 3Day2()//Winning goal by Lionel Messi//Two corner kicksLine 4output \\"Three throw-ins\\"//Three throw-ins",\n    "code_snippet": "action Day3()\\n    Day1()\\n    Day2()\\n    output \\"Three throw-ins\\"\\nend\\n\\naction Day2()\\n    Day1()\\n    output \\"Two corner kicks\\"\\nend\\n\\naction Day1()\\n    output \\"Winning goal by Lionel Messi\\"\\nend"\n}'},
 {'MQCode': '3A17',
  'learning_objective': 'Systematically test and refine programs using a range of test cases.',
  'topic': 'Temperature classification',
  'question_str': '{\n    "Question": "Surfer is coding a procedure that will take a temperature in Celsius as an parameter and print out whether water will become ice ( <= 0\\ufffd ), boil ( >= 100\\ufffd) or remain water.   The code for Surfer\'s procedure is as follows:For which of the following calls to waterState() will Surfer\'s procedure output misleading results?",\n    "Answer1": "waterState(120)",\n    "Answer2": "waterState(50)",\n    "Answer3": "waterState(0)",\n    "Answer4": "waterState(-5)",\n    "CorrectAnswer": "1",\n    "Explanation": "At a temperature of 120 degrees Celsius, waterState() will output both the phrase \\"Water will boil\\" AND the phrase \\"Water will not freeze or boil\\".   waterState() should use a single if \\ufffd elseif \\ufffd else conditional for the best results. ",\n    "code_snippet": "action waterState(integer temp)\\n    if temp >= 100\\n        output \\"Water will boil\\"\\n    end\\n    if temp <= 0\\n        output \\"Water will freeze\\"\\n    else\\n        output \\"Water will not freeze or boil\\"\\n    end\\nend"\n}'}
]

Define interest area categories and subtopics.

In [14]:
# current interest areas
interest_areas = [
    "World History",
    "Business",
    "Sports and Athletics",
    "Science and engineering",
    "Everyday Life/School Life",
    "Social justice", 
    "Diversity and inclusion",
    "Agriculture",
    "Math",
    "Music",
    "Art and Design",
    "(Video) Gaming",
]

In [15]:
# dictionary of subtopics within each interest area
interests = {
    "World History": [
        "Ancient Civilizations",
        "Classical Empires (e.g., Roman, Greek, Persian)",
        "Middle Ages and Feudalism",
        "Renaissance and Enlightenment",
        "Colonialism and Imperialism",
        "World Wars (WWI and WWII)",
        "Cold War and the Space Race",
        "Decolonization and Independence Movements",
        "Globalization and the Modern World",
        "Revolutionary Movements and Social Change",
    ],
    "Business": [
        "Entrepreneurship and Startups",
        "Business Strategy and Planning",
        "Marketing and Advertising",
        "Finance and Accounting",
        "Supply Chain Management",
        "E-commerce and Online Business",
        "Human Resources and Talent Management",
        "Business Ethics and Corporate Social Responsibility",
        "International Business and Globalization",
        "Business Innovation and Technology",
    ],
    "Sports and Athletics": [
        "Football (Soccer)",
        "Basketball",
        "Weight lifting",
        "Tennis",
        "Running and Track Events",
        "Gymnastics",
        "Swimming and Aquatics",
        "Team Sports",
        "Extreme Sports (e.g., Surfing, Skateboarding)",
        "Sports Psychology and Performance",
    ],
    "Science and engineering": [
        "Physics",
        "Chemistry",
        "Biology",
        "Archeology",
        "Electrical Engineering",
        "Mechanical Engineering",
        "Civil Engineering",
        "Environmental Science",
        "Astronomy and Astrophysics",
        "Genetics and Biotechnology",
    ],
    "Everyday Life/School Life": [
        "Time Management and Organization",
        "Study Skills and Techniques",
        "Effective Communication",
        "Goal Setting and Motivation",
        "Mental/Physical Health and Wellness",
        "Relationships and Social Skills",
        "Personal Finance and Budgeting",
        "Technology and Digital Skills",
        "Creativity and Hobbies",
        "Stress Management and Self-Care",
        "Public Speaking and Presentation Skills",
        "Career Planning and Job Search",
        "Mindfulness and Meditation",
        "Conflict Resolution and Negotiation",
        "Self-Reflection and Personal Growth",
    ],
    "Social justice": [
        "Human Rights",
        "Racial Justice",
        "Gender Equality",
        "LGBTQ+ Rights",
        "Immigrant Rights",
        "Environmental Justice",
        "Disability Rights",
        "Criminal Justice Reform",
        "Social Inequality and Poverty",
        "Indigenous Rights and Sovereignty",
        "Income inequality and economic justice",
        "Educational equity and access",
        "Homelessness",
        "Healthcare access and affordability",
        "Religious freedom and tolerance",
        "Worker\'s rights and fair labor practices",
        "Fair and affordable housing",
        "Animal rights",
        "Net neutrality and digital privacy",
    ],
    "Diversity and inclusion": [
        "Cultural Competence",
        "Implicit Bias",
        "Intersectionality",
        "Ethnic and Racial Diversity",
        "Gender Diversity",
        "Religious Diversity",
        "Disability Inclusion",
        "Age Inclusivity",
        "LGBTQ+ Inclusion",
        "Inclusive Leadership",
    ],
    "Agriculture": [
        "Crop Production",
        "Livestock Farming",
        "Sustainable Agriculture",
        "Organic Farming",
        "Precision Agriculture",
        "Agribusiness and Farm Management",
        "Food Security and Hunger",
        "Crop Protection and Pest Management",
        "Soil Science and Fertility",
        "Agricultural Technology and Innovation",
    ],
    "Math": [
        "Algebra",
        "Geometry",
        "Trigonometry",
        "Probability and Statistics",
        "Functions and Graphs",
        "Exponents and Logarithms",
        "Quadratic Equations",
    ],
    "Music": [
        "Music Theory",
        "Music History",
        "Instrumental Techniques",
        "Vocal Techniques",
        "Music Composition",
        "Music Performance",
        "Music Notation",
        "Music Technology",
        "Music in Film and Media",
        "Music Therapy",
    ],
    "Art and Design": [
        "Drawing and Sketching",
        "Painting Techniques",
        "Sculpture and 3D Design",
        "Graphic Design",
        "Typography",
        "Photography and Photo Editing",
        "Fashion Design",
        "Illustration",
        "Art History",
        "Digital Art and Animation",
    ],
    "(Video) Gaming": [
        "Game Development",
        "Game Design",
        "Game Programming",
        "Game Art and Animation",
        "Game Sound Design",
        "Game Testing and Quality Assurance",
        "Game Marketing and Publishing",
        "Game Accessibility",
        "Esports and Competitive Gaming",
        "Gaming Communities and Online Platforms",
    ],
}

In [16]:
# categorize examples by learning objective group
ex_groups = {
    'C': [],
    'N': [],
    'D': [],
    'A': [],
    'I': [],
}

for ex in examples:
    ex_groups[ex['MQCode'][1]].append(ex)

Create Few-Shot Example Question Template

In [17]:
# Create custom example selector that chooses examples from the same 
# learning objective group as the target learning objective

class CustomExampleSelector(BaseExampleSelector):
    def __init__(self, examples: List[Dict[str, str]]):
        self.examples = examples
    def add_example(self, example: Dict[str, str]) -> None:
        # Add new example to store for a key.
        self.examples.append(example)
    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        # Select which examples to use based on the inputs.
        for code, obj in lo_list.items():
            if obj == input_variables['learning_objective']:
                similar_exs = ex_groups[code[1]]
                few_exs = random.sample(similar_exs, min(len(similar_exs), 3))
                return few_exs

In [220]:
# Instruct GPT-4 Few-Shot Example Template

question_template = """Learning objective: {learning_objective}\nTopic: {topic}\nQuestion:\n```json\n{{{question_str}}}\n```"""

example_prompt = PromptTemplate(
    input_variables=["learning_objective", "topic", "question_str"],
    template=question_template,
)

def make_prompt(coding: bool = False) -> FewShotPromptTemplate:
    if coding:
        exs = random.sample(coding_q_exs, min(len(coding_q_exs), 3))
        few_shot_prompt = FewShotPromptTemplate(
            examples=exs,
            example_prompt=example_prompt,
            prefix="""System Message: You are a high school computer science teacher who wants to connect CS concepts to the diverse interests of your students so that they can better see themselves working in CS. Applications of CS to different fields of study often have similar solutions in code and teach the same underlying CS principles. And by seeing questions related to a wide variety of disciplines, students can better appreciate that CS is everywhere, for everyone, and that the essence of good computer science is to find the common patterns in problems from all aspects of life and to develop solutions for them.\n\nTask: Given a CS learning objective and a topic of interest to your students, generate a question in JSON format that applies the learning objective to a scenario related to the provided topic. The question should involve a code snippet specified in the quorum programming language as shown in the example questions.""",    
            suffix="Learning objective: {learning_objective}\nTopic: {topic}\nQuestion:",
            input_variables=["learning_objective", "topic"]
        )
    else:
        example_selector = CustomExampleSelector(examples)
        few_shot_prompt = FewShotPromptTemplate(
            example_selector=example_selector,
            example_prompt=example_prompt,
            prefix="""System Message: You are a high school computer science teacher who wants to connect CS concepts to the diverse interests of your students so that they can better see themselves working in CS. Applications of CS to different fields of study often have similar solutions in code and teach the same underlying CS principles. And by seeing questions related to a wide variety of disciplines, students can better appreciate that CS is everywhere, for everyone, and that the essence of good computer science is to find the common patterns in problems from all aspects of life and to develop solutions for them.\n\nTask: Given a CS learning objective and a topic of interest to your students, generate a question in JSON format that applies the learning objective to a scenario related to the provided topic. Specify the content of any charts or graphics referenced in your question.""",    
            suffix="Learning objective: {learning_objective}\nTopic: {topic}\nQuestion:",
            input_variables=["learning_objective", "topic"]
        )
    return few_shot_prompt


Generate Questions.

In [226]:
def generate_qs(folder_path: str, coding: bool = False) -> list[dict]:
    # generate questions for each interest and learning objective group
    generated_qs = []
    for area, topic_list in list(interests.items()):
        for topic in topic_list[:1]:
            # randomly select learning standard
            lo_code = random.choice(random.choice(list(lo_code_groups.items()))[1])
            obj = lo_list[lo_code]
            #print(f"Interest: {topic}\nLearning Objective: {lo_code}. {obj}\n")
            
            # create prompt
            prompt = make_prompt(coding).format(learning_objective=obj, topic=topic)
            print(prompt)

            # get gpt-4 response
            res = llm(prompt)
            q_str = "{" + res.split("{")[-1].split("}")[0] + "}"
            print(q_str + "\n")
            q = {
                'learning_objective': obj,
                'interest_area': area,
                'topic': topic,
                'MQCode': lo_code,
                'question_str': q_str,
            }
            generated_qs.append(q)

            # output question json to file
            f_name = q['MQCode'] + "-" + q['topic'].replace(" ", "_")
            with open(folder_path + f_name, 'w') as f:
                json.dump(q, fp=f, indent=4)

    return generated_qs


In [227]:
# generate questions
coding = True
folder_path = "coding_questions\\6-28-2023\\"
qs = generate_qs(folder_path, coding)

System Message: You are a high school computer science teacher who wants to connect CS concepts to the diverse interests of your students so that they can better see themselves working in CS. Applications of CS to different fields of study often have similar solutions in code and teach the same underlying CS principles. And by seeing questions related to a wide variety of disciplines, students can better appreciate that CS is everywhere, for everyone, and that the essence of good computer science is to find the common patterns in problems from all aspects of life and to develop solutions for them.

Task: Given a CS learning objective and a topic of interest to your students, generate a question in JSON format that applies the learning objective to a scenario related to the provided topic. The question should involve a code snippet specified in the quorum programming language as shown in the example questions.

Learning objective: Describe choices made during program development using c

In [236]:
# check a code snipped
q_idx = 0
print(json.loads(qs[q_idx]['question_str'])['code_snippet'])

number[] egyptDynasties = new number[31]
for (i = 0, i <= 30, i++)
    egyptDynasties[i] = assignDynastyYear(i)
end
